ASAP CRN Prototype workflow 

# ASAP CRN Prototype workflow 

Skeleton workflow to begin development of a unified workflow for _all_ ASAP CRN single cell (or nucleus) RNA sequence data.
based on:
* PD_MFG_snRNAseq_with_redos.Rmd from Team Lee ***sequence alignment AND processing***
* Harmony-RNA-Workflow [snakefile repo](https://github.com/shahrozeabbas/Harmony-RNA-Workflow) ***processing only***
  
28 July 2023
Andy Henrie


Note that the 


# TOOLS / PACKAGES


# STEPS

## housekeeping
- define names
- list of fastq
- collect meta-data


## alignment 
- cellranger

## preprocessing
- convert to data object (e.g. Scanpy, Surat)
- add metadata
    - from metadata
    - calculated
        - doublets
        - mt
        - rb

## QC
- filtering
    - mt
    - doublets
    - counts
    - features

## processing
- normalize
- identify highly variable genes
- scale
- assign
    - cell cycle scores

## harmonize (LATER)
- concatenate
- RunHarmony


## HOUSEKEEPING

In [ ]:
project = "asap_crn_XXX_snRNAseq"

input_table = "input_table.csv"


## import meta-data 


## merge tables



#---------- QC Filtering
#----- Filtering

            # nCount_RNA %between% c(500, 2e5) &
            # nFeature_RNA %between% c(500, 12500) &
            # percent.mt < 0.5 & percent.rb < 0.5,

nUMI_low = 1000    # harmony = 500
nUMI_high = 20000  # harmony = 20000
nGene_low = 700    # harmony = 500
nGene_high = 6000  # harmony = 12500
percent_mito_high = 5  # 
percent_ribo_high = 30
log10GenesPerUMI_low = 0.85




## ALIGNMENT

> SOURCE: [gs://asap-raw-data-team-lee/scripts/PD_MFG_snRNAseq_with_redos.Rmd](gs://asap-raw-data-team-lee/scripts/PD_MFG_snRNAseq_with_redos.Rmd)


### Cellranger

Transfer sequencing data from genomics to rawdata  
cellranger-6.0.1 to align and count  
performed mRNA alignment for standard single cell counts  
also performed mRNA and premRNA (introns) counts for trajectory analysis  

In [ ]:
%%bash
#---------- cellranger
#----- mRNA alignment and counting
# used for single cell
# don't use --include-introns

#---------- cellranger_mRNA_premRNA
#----- mRNA and premRNA alignment and counting
# used for single nuclei
module load cellranger/cellranger-6.0.1


"cellranger count  
  --include-introns 
  --id=SAMPLE_count 
  --transcriptome=/secondary/projects/bras/primary/software/cellranger_v6/refdata-gex-GRCh38-2020-A 
  --fastqs=../fastq 
  --sample=SAMPLE 
  --localcores=4"
  

In [1]:
import pandas as pd


In [4]:

data_path = "../data/team-lee/metadata"
HIP_covar = pd.read_csv(f"{data_path}/HIP/covar.csv")
HIP_cases = pd.read_csv(f"{data_path}/HIP/PD_ASAP_Sample_batch_information_banner_cases.csv")
HIP_control = pd.read_csv(f"{data_path}/HIP/PD_ASAP_Sample_batch_information_banner_controls.csv")

MFG_covar = pd.read_csv(f"{data_path}/MFG/covar.csv")
MFG_cases = pd.read_csv(f"{data_path}/MFG/PD_ASAP_Sample_batch_information_banner_cases.csv")
MFG_control = pd.read_csv(f"{data_path}/MFG/PD_ASAP_Sample_batch_information_banner_controls.csv")


SN_covar = pd.read_csv(f"{data_path}/SN/covar.csv")
SN_cases = pd.read_csv(f"{data_path}/SN/PD_ASAP_Sample_batch_information_banner_cases.csv")
SN_control = pd.read_csv(f"{data_path}/SN/PD_ASAP_Sample_batch_information_banner_controls.csv")


In [9]:
HIP_covar.head()


,BRAIN_REGION,SAMPLE,COUNT_ID,SAMPLE_FULL,ID,SEQ_ID,BATCH,GROUP,DIAGNOSIS,SEX,AGE,DIAGNOSIS_AGE,DIAGNOSIS_YEARS,DISEASE,ALZHEIMERS_BRAAK,DEMENTIA,DEMENTIA_AGE,DEMENTIA_YEARS,WEIGHT_USED,TOTAL_WEIGHT
0,Hippocampus,HC_2067,HIP_HC_2067,HIP_HC_20-67,HC_20-67,HIP2067HC,BATCH_8,HC,0,1,72,NaN,NaN,HC,I,0,NaN,NaN,35,53
1,Hippocampus,HC_1939,HIP_HC_1939,HIP_HC_19-39,HC_19-39,HIP1939HC,BATCH_8,HC,0,2,93,NaN,NaN,HC,II,0,NaN,NaN,29,49
2,Hippocampus,PD_0413,HIP_PD_0413,HIP_PD_04-13,PD_04-13,HIP0413PD,BATCH_8,PD,1,1,72,62.0,10.0,PD,I,0,NaN,NaN,33,49
3,Hippocampus,PD_1504,HIP_PD_1504,HIP_PD_15-04,PD_15-04,HIP1504PD,BATCH_8,PD,1,2,72,54.0,18.0,PD,II,1,68.0,4.0,35,62
4,Hippocampus,HC_1225,HIP_HC_1225,HIP_HC_12-25,HC_12-25,HIP1225HC,BATCH_9,HC,0,1,80,NaN,NaN,HC,IV,0,NaN,NaN,19,19


In [10]:
HIP_control.head()

,CaseID,race,gender,expired_age,PMI,last_mmse_test_score,last_mmse_interval_before_death,motor_updrs_score_on,motor_updrs_score_off,motor_updrs_score_months_prior_to_death,...,nctx_frontal,sum_lb_density,nctx_parietal,PathDXSummary,neurological_dx,neurological_dx_years,neurological_dx_age,dementia_yn,dementia_years,demential_age
0,18-64,1,1,72,4.60,29.0,18.0,NaN,NaN,NaN,...,0,0,0,"Control (history); Neurofibrillary tangles, me...",Control,NaN,NaN,0,NaN,NaN
1,20-57,1,1,81,4.58,28.0,8.0,NaN,4.0,8.0,...,0,0,0,Control; Neurofibrillary tangles and non-speci...,Control,NaN,NaN,0,NaN,NaN
2,20-61,1,2,96,2.83,27.0,26.0,NaN,12.0,26.0,...,0,0,0,Cognitive impairment without dementia (history...,Mild Cognitive Impairment,2.0,94.0,0,NaN,NaN
3,20-62,1,1,81,4.42,28.0,12.0,NaN,14.0,12.0,...,0,0,0,Cognitively normal control (history); Cerebral...,Control,NaN,NaN,0,NaN,NaN
4,20-67,1,1,72,5.25,NaN,NaN,NaN,NaN,NaN,...,0,0,0,Control (history); Microscopic changes of Alzh...,Control,NaN,NaN,0,NaN,NaN


In [11]:
HIP_cases.head()

,CaseID,race,gender,expired_age,PMI,last_mmse_test_score,last_mmse_interval_before_death,motor_updrs_score_on,motor_updrs_score_off,motor_updrs_score_months_prior_to_death,...,nctx_temporal,nctx_frontal,sum_lb_density,nctx_parietal,PathDXSummary,neurological_dx,neurological_dx_age,dementia_yn,dementia_years,demential_age
0,00-09,1.0,1.0,64.0,4.00,22.0,34.0,NaN,NaN,NaN,...,1.0,1.0,19.0,1.0,Parkinson's disease; Dementia (history); Charc...,Parkinson's disease,49.0,1.0,6.0,58.0
1,03-48,1.0,1.0,90.0,3.00,27.0,26.0,NaN,NaN,NaN,...,1.0,1.0,16.0,1.0,Parkinson's disease; microscopic lesions of Al...,Parkinson's disease,75.0,0.0,NaN,NaN
2,04-13,1.0,1.0,72.0,2.00,NaN,NaN,NaN,17.5,1.0,...,1.0,1.0,11.0,0.0,"Parkinson’s disease; Astrocytoma, grade IV (Da...",Parkinson's disease,62.0,0.0,NaN,NaN
3,06-02,1.0,1.0,84.0,2.66,26.0,25.0,NaN,7.0,2.0,...,0.0,0.0,11.0,0.0,Control; Mild cognitive impairment (history); ...,Mild Cognitive Impairment,81.0,0.0,NaN,NaN
4,12-25,1.0,1.0,80.0,3.50,29.0,5.0,NaN,0.0,5.0,...,0.0,0.0,3.0,0.0,"Control; Neurofibrillary tangles, argyrophilic...",Control,NaN,0.0,NaN,NaN


# PREPROCESSING

> SOURCE: [Harmony-RNA-Workflow/scripts/main/preprocess.R](https://github.com/shahrozeabbas/Harmony-RNA-Workflow/blob/main/scripts/main/preprocess.R)


This step creates a suitable data object including the relavent metadata.

1. convert raw cellranger counts to data objects.  e.g. Seurat object as per examples. Could be scanpy anndata object (preferred?)
2. add metadata
    - collect metadata - sample IDs, conditions, etc.
    - calculate features/metrics - doublets, mt, rb
    


In [ ]:
# TEAM LEE
#---------- Seurat_Object
#----- Count Matrix
seurat_counts <- lapply(NamesCount, function(Name){
  count <- Read10X(data.dir = paste0(DirCounts,
                                     Name,
                                     "_count/outs/filtered_feature_bc_matrix/"),
                   gene.column=2, # col1 Ensemble, col2 Symbol
                   unique.features=TRUE,
                   strip.suffix=TRUE) 
  seurat_obj <- CreateSeuratObject(counts = count, 
                                   project = Name,
                                   min.cells = 10,
                                   min.genes = 200)
})
# Make sound to notify that evaluation is complete.
beepr::beep()
#----- Merge Seurat Objects
seurat_combined <- merge(x = seurat_counts[[1]], 
                         y = seurat_counts[2:length(seurat_counts)],
                         add.cell.ids = unlist(NamesCount), 
                         project = NameProject)

#---------- Add Metadata
#----- Add Percentage of Mitocondrial and Ribosomal Counts
seurat_combined <- Add_Mito_Ribo_Seurat(seurat_object = seurat_combined, 
                                        species = "Human")
# View(seurat_combined@meta.data)
# orig.ident: contains the sample identity if known
# nCount_RNA: number of UMIs per cell
# nFeature_RNA: number of genes detected per cell
seurat_meta <- function(seurat_obj, name_count) {
  #----- Create metadata dataframe
  metadata <- seurat_obj@meta.data
  #----- Rename 
  names(metadata)[1:3] <- c(name_count,"nUMI","nGene")
  #----- Add Cell IDs
  metadata$cells <- rownames(metadata)
  #----- Add number of genes per UMI
  metadata$log10GenesPerUMI <- 
    log10(metadata$nGene) / log10(metadata$nUMI)
  #----- Compute percent mito ratio
  # metadata <- cbind(metadata,
  #                   PercentageFeatureSet(object = seurat_combined, 
  #                                        pattern = "^MT-") / 100)
  # names(metadata)[6] <- "mitoRatio"
  #----- Merge Covar
  metadata <- metadata %>% 
                left_join(covar, by = name_count)
  rownames(metadata) <- metadata$cells
  #----- Add metadata back to Seurat object
  seurat_obj@meta.data <- metadata
  return(seurat_obj)
}
seurat_combined <- seurat_meta(seurat_obj = seurat_combined,
                                name_count = NameCount)


#---------- Senescence
#----- Senescence Markers, p16 = CDKN2A, p21 = CDKN1A
#----- + extra senescence-relevant genes of interest: p19 = CDKN2D, uPAR = PLAUR
seurat_combined$CDKN2A_Count <- 
  seurat_combined@assays$RNA@counts[rownames(seurat_combined) == "CDKN2A",]
seurat_combined$CDKN1A_Count <- 
  seurat_combined@assays$RNA@counts[rownames(seurat_combined) == "CDKN1A",]
seurat_combined$CDKN2D_Count <- 
  seurat_combined@assays$RNA@counts[rownames(seurat_combined) == "CDKN2D",]
seurat_combined$GPNMB_Count <- 
  seurat_combined@assays$RNA@counts[rownames(seurat_combined) == "GPNMB",]
seurat_combined$PLAUR_Count <- 
  seurat_combined@assays$RNA@counts[rownames(seurat_combined) == "PLAUR",]

#---------- Save Data
#----- Remove Data
rm(seurat_counts)
#----- Save Data
saveRDS(seurat_combined, file="seurat_results/seurat_combined.rds")

## QC 

> SOURCE: [Harmony-RNA-Workflow/scripts/main/gmm_doublet_calling.R](https://github.com/shahrozeabbas/Harmony-RNA-Workflow/blob/main/scripts/main/gmm_doublet_calling.R) and [Harmony-RNA-Workflow/scripts/main/filter.R](https://github.com/shahrozeabbas/Harmony-RNA-Workflow/blob/main/scripts/main/filter.R)


Quality control thresholds calculated/set.   
Probably we should simply Add QC features (columns) instead of removing entries at this point.

We also can generate figure artifacts.

e.g. Senescence Cell not selectively removed  

In [ ]:
# TEAM LEE
#---------- Seurat_QC_Figures
#----- Load data
seurat_combined <- readRDS(file = "seurat_results/seurat_combined.rds")

#---------- Cell Counts
#----- Number of Cells per Sample
plot <- ggSample + 
  geom_bar(data = seurat_combined@meta.data, 
           aes(x=SAMPLE, fill=SAMPLE)) + 
  ggtitle("Number of Cells") + 
  theme(axis.title.x = element_blank()) + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Cells_sample.png", 
    width=600, 
    height=400)
plot
dev.off()
#----- Number of Cells per Group
plot <- ggGroup + 
  geom_boxplot(data = seurat_combined@meta.data %>% 
                        dplyr::count(GROUP, SAMPLE) %>% 
                        dplyr::rename(Count = n), 
           aes(x=GROUP, y=Count, fill=GROUP)) + 
  ggtitle("Number of Cells") + 
  theme(axis.title.x = element_blank()) + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Cells_group.png", 
    width=600, 
    height=400)
plot
dev.off()

#---------- UMI Transcript Counts 
#----- Number of Transcripts per cell per Sample
plot <- ggSample + 
  geom_density(data = seurat_combined@meta.data, 
               alpha = 0.2,
               aes(x=nUMI, fill=SAMPLE, color=SAMPLE)) + 
  ggtitle("Number of Transcripts per Cell") + 
  ylab("Cell density") + 
  geom_vline(xintercept = nUMI_low, linetype="dashed", colour = "red3") + 
  geom_vline(xintercept = nUMI_high, linetype="dashed", colour = "red3") + 
  scale_x_log10() + 
  facet_wrap(~SAMPLE) +
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Transcripts_per_Cell_sample.png", 
    width=1000, 
    height=1000)
plot
dev.off()
#----- Number of Transcripts per cell per Group
plot <- ggGroup + 
  geom_density(data = seurat_combined@meta.data, 
               alpha = 0.2,
               aes(x=nUMI, fill=GROUP, color=GROUP)) + 
  ggtitle("Number of Transcripts per Cell") + 
  ylab("Cell density") + 
  geom_vline(xintercept = nUMI_low, linetype="dashed", colour = "red3") + 
  geom_vline(xintercept = nUMI_high, linetype="dashed", colour = "red3") + 
  scale_x_log10()
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Transcripts_per_Cell_group.png", 
    width=600, 
    height=400)
plot
dev.off()

#---------- Gene Counts
#----- Number of Genes per cell per Sample
plot <- ggSample + 
  geom_density(data = seurat_combined@meta.data, 
               alpha = 0.2,
               aes(x=nGene, fill=SAMPLE, color=SAMPLE)) + 
  ggtitle("Number of Genes per Cell") + 
  ylab("Cell density") + 
  geom_vline(xintercept = nGene_low, linetype="dashed", colour = "red3") + 
  geom_vline(xintercept = nGene_high, linetype="dashed", colour = "red3") + 
  scale_x_log10() + 
  facet_wrap(~SAMPLE) + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Genes_per_Cell_sample.png", 
    width=1000, 
    height=1000)
plot
dev.off()
#----- Number of Genes per cell per Group
plot <- ggGroup + 
  geom_density(data = seurat_combined@meta.data, 
               alpha = 0.2,
               aes(x=nGene, fill=GROUP, color=GROUP)) + 
  ggtitle("Number of Genes per Cell") + 
  ylab("Cell density") + 
  geom_vline(xintercept = nGene_low, linetype="dashed", colour = "red3") + 
  geom_vline(xintercept = nGene_high, linetype="dashed", colour = "red3") + 
  scale_x_log10()
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Genes_per_Cell_group.png", 
    width=600, 
    height=400)
plot
dev.off()


#----- Number of Genes per cell per Sample
plot <- ggSample + 
  geom_boxplot(data = seurat_combined@meta.data,
               aes(x=SAMPLE, y=log10(nGene), fill=SAMPLE)) + 
  ggtitle("Number of Genes per Cell") + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Genes_per_Cell_per_Sample.png", 
    width=600, 
    height=400)
plot
dev.off()
#----- Number of Genes per cell per Group
plot <- ggGroup + 
  geom_boxplot(data = seurat_combined@meta.data,
               aes(x=GROUP, y=log10(nGene), fill=GROUP)) + 
  ggtitle("Number of Genes per Cell") + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Genes_per_Cell_per_Group.png", 
    width=600, 
    height=400)
plot
dev.off()

#---------- Mitochondria Percentage
#----- Mitochondrial Transcripts per cell per Sample
lowest_mito <- min(seurat_combined@meta.data$percent_mito[seurat_combined@meta.data$percent_mito > 0])
plot <- ggSample + 
  geom_density(data = seurat_combined@meta.data %>% 
                 # handle the log 0 -- let these show up on plot 
                 # (replace 0s with the min non-zero value)
                 mutate(percent_mito = case_when(percent_mito == 0 ~ lowest_mito,
                                                 TRUE ~ percent_mito)), 
               alpha = 0.2,
               aes(x=percent_mito, fill=SAMPLE, color=SAMPLE)) + 
  ggtitle("Mitochondrial Transcripts per Cell") + 
  ylab("Percentage") + 
  geom_vline(xintercept = percent_mito_high, linetype="dashed", colour = "red3") + 
  scale_x_log10() + 
  facet_wrap(~SAMPLE) + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Mitochondrial_Transcripts_per_Cell_sample.png", 
    width=1000, 
    height=1000)
plot
dev.off()
#----- Mitochondrial Transcripts per cell per Group
plot <- ggGroup + 
  geom_density(data = seurat_combined@meta.data %>% 
                 # handle the log 0 -- let these show up on plot 
                 # (replace 0s with the min non-zero value)
                 mutate(percent_mito = case_when(percent_mito == 0 ~ lowest_mito,
                                                 TRUE ~ percent_mito)), 
               alpha = 0.2,
               aes(x=percent_mito, fill=GROUP, color=GROUP)) + 
  ggtitle("Mitochondrial Transcripts per Cell") + 
  ylab("Percentage") + 
  geom_vline(xintercept = percent_mito_high, linetype="dashed", colour = "red3") + 
  scale_x_log10()
plot
png(file="seurat_results/figures/Seurat_QC_Mitochondrial_Transcripts_per_Cell_group.png", 
    width=600, 
    height=400)
plot
dev.off()
rm(lowest_mito)

#---------- Ribosomal Percentage
#----- Ribosomal Transcripts per cell per Sample
lowest_ribo <- min(seurat_combined@meta.data$percent_ribo[seurat_combined@meta.data$percent_ribo > 0])
plot <- ggSample + 
  geom_density(data = seurat_combined@meta.data %>% 
                 # handle the log 0 -- let these show up on plot 
                 # (replace 0s with the min non-zero value)
                 mutate(percent_ribo = case_when(percent_ribo == 0 ~ lowest_ribo,
                                                 TRUE ~ percent_ribo)), 
               alpha = 0.2,
               aes(x=percent_ribo, fill=SAMPLE, color=SAMPLE)) + 
  ggtitle("Ribosomal Transcripts per Cell") + 
  ylab("Percentage") + 
  geom_vline(xintercept = percent_ribo_high, linetype="dashed", colour = "red3") + 
  scale_x_log10() + 
  facet_wrap(~SAMPLE) + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Ribosomal_Transcripts_per_Cell_sample.png", 
    width=1000, 
    height=1000)
plot
dev.off()
#----- Ribosomal Transcripts per cell per Group
plot <- ggGroup + 
  geom_density(data = seurat_combined@meta.data %>% 
                 # handle the log 0 -- let these show up on plot 
                 # (replace 0s with the min non-zero value)
                 mutate(percent_ribo = case_when(percent_ribo == 0 ~ lowest_ribo,
                                                 TRUE ~ percent_ribo)), 
               alpha = 0.2,
               aes(x=percent_ribo, fill=GROUP, color=GROUP)) + 
  ggtitle("Ribosomal Transcripts per Cell") + 
  ylab("Percentage") + 
  geom_vline(xintercept = percent_ribo_high, linetype="dashed", colour = "red3") + 
  scale_x_log10()
plot
png(file="seurat_results/figures/Seurat_QC_Ribosomal_Transcripts_per_Cell_group.png", 
    width=600, 
    height=400)
plot
dev.off()
rm(lowest_ribo)

#---------- Complexity
#----- Complexity Genes per Transcript per cell per Sample
plot <- ggSample + 
  geom_density(data = seurat_combined@meta.data, 
               alpha = 0.2,
               aes(x=log10GenesPerUMI, fill=SAMPLE, color=SAMPLE)) + 
  ggtitle("Complexity Genes per Transcript per Cell") + 
  ylab("Cell density") + 
  geom_vline(xintercept = log10GenesPerUMI_low, 
             linetype = "dashed", 
             colour = "red3") + 
  facet_wrap(~SAMPLE) + 
  theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_QC_Complexity_Genes_per_Transcript_per_Cell_sample.png", 
    width=1000, 
    height=1000)
plot
dev.off()
#----- Complexity Genes per Transcript per cell per Group
plot <- ggGroup + 
  geom_density(data = seurat_combined@meta.data, 
               alpha = 0.2,
               aes(x=log10GenesPerUMI, fill=GROUP, color=GROUP)) + 
  ggtitle("Complexity Genes per Transcript per Cell") + 
  ylab("Cell density") + 
  geom_vline(xintercept = log10GenesPerUMI_low, 
             linetype = "dashed", 
             colour = "red3")
plot
png(file="seurat_results/figures/Seurat_QC_Complexity_Genes_per_Transcript_per_Cell_group.png", 
    width=600, 
    height=400)
plot
dev.off()

#---------- UMI Transcript Counts vs Gene Counts
#----- Number of Transcripts vs Number of Genes per cell per Sample
plot <- ggSample + 
  geom_point(data = seurat_combined@meta.data, 
               aes(x=nUMI, y=nGene, color=percent_mito),
               size=0.1) + 
  scale_colour_viridis_c() +
  ggtitle("Number of Transcripts vs Gene Counts per Cell") + 
  geom_vline(xintercept = nUMI_low, linetype="dashed", colour = "red3") + 
  geom_vline(xintercept = nUMI_high, linetype="dashed", colour = "red3") + 
  geom_hline(yintercept = nGene_low, linetype="dashed", colour = "red3") + 
  geom_hline(yintercept = nGene_high, linetype="dashed", colour = "red3") +  
  scale_x_log10() + 
  scale_y_log10() + 
  stat_smooth(method=lm) + 
  facet_wrap(~SAMPLE)
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Transcripts_vs_Genes_per_Cell_sample.png", 
    width=1000, 
    height=1000)
plot
dev.off()
#----- Number of Transcripts vs Number of Genes per cell per Group
plot <- ggSample + 
  geom_point(data = seurat_combined@meta.data, 
               aes(x=nUMI, y=nGene, color=percent_mito),
               size=0.1) + 
  scale_colour_viridis_c() +
  ggtitle("Number of Transcripts vs Gene Counts per Cell") + 
  geom_vline(xintercept = nUMI_low, linetype="dashed", colour = "red3") + 
  geom_vline(xintercept = nUMI_high, linetype="dashed", colour = "red3") + 
  geom_hline(yintercept = nGene_low, linetype="dashed", colour = "red3") + 
  geom_hline(yintercept = nGene_high, linetype="dashed", colour = "red3") +  
  scale_x_log10() + 
  scale_y_log10() + 
  stat_smooth(method=lm) + 
  facet_wrap(~GROUP)
plot
png(file="seurat_results/figures/Seurat_QC_Number_of_Transcripts_vs_Genes_per_Cell_group.png", 
    width=600, 
    height=400)
plot
dev.off()

In [ ]:
#---------- Senescence Markers 
#----- Number of Transcripts or Genes vs Senescence Marker per Cell sample
plot_qc_senescence_markers <- function(senescence_gene, qc_metric) {
  #----- check arguments: senescence_gene
  possible_genes <- gsub("_Count", "",
                         names(seurat_combined@meta.data[endsWith(names(seurat_combined@meta.data), "_Count")]))
  if (!(senescence_gene %in% possible_genes)) {
    stop(paste("senescence_gene must be one of:", 
               paste(possible_genes, collapse = ", "),
               "\nYou provided: senescence_gene =", senescence_gene)
         )
  }
  #----- check arguments: qc_metric
  if (qc_metric == "Transcripts") {
    vline_variable <- "nUMI"
  } else if (qc_metric == "Genes") {
    vline_variable <- "nGene"
  } else {
    stop(paste("qc_metric must either be 'Transcripts' or 'Genes'.\n",
               "You provided: qc_metric =", qc_metric))
  }
  #----- make plot
  plot <- ggSample + 
    geom_point(data = seurat_combined@meta.data, 
                 aes(x=nUMI, y=get(paste0(senescence_gene, "_Count")), color=percent_mito),
                 size=0.1) + 
    scale_colour_viridis_c() +
    ggtitle(paste("Number of", qc_metric, "vs", senescence_gene, "per Cell")) + 
    labs(y = paste0(senescence_gene, "_Count")) +
    geom_vline(xintercept = get(paste0(vline_variable, "_low")), linetype="dashed", colour = "red3") + 
    geom_vline(xintercept = get(paste0(vline_variable, "_high")), linetype="dashed", colour = "red3") + 
    scale_x_log10() + 
    facet_wrap(~SAMPLE)
  print(plot)
  png(file=paste0("seurat_results/figures/Seurat_QC_Number_of_", qc_metric,
                  "_vs_", senescence_gene, "_per_Cell_sample.png"), 
      width=1000, 
      height=1000)
  print(plot)
  dev.off()
} 

# loop through all senescence genes, plotting transcripts & genes:
senescence_genes <- gsub("_Count", "", names(seurat_combined@meta.data[endsWith(names(seurat_combined@meta.data), "_Count")]))
for (gene in senescence_genes) {
  print(gene)
  plot_qc_senescence_markers(gene, "Transcripts")
  plot_qc_senescence_markers(gene, "Genes")
}

#----- Simple lm of senescence ~ age.
p16_metadata <- seurat_combined@meta.data %>%
  mutate(CDKN2A_Positive = case_when(CDKN2A_Count > 0 ~ TRUE,
                                     CDKN2A_Count == 0 ~ FALSE)) %>%
  group_by(SAMPLE, CaseID) %>%
  summarize(total_cells = n(),
            total_p16_transcripts = sum(CDKN2A_Count),
            total_p16_cells = sum(CDKN2A_Positive),
            prop_p16_cells = total_p16_cells/total_cells) %>%
  left_join(y = covar, by = c("CaseID", "SAMPLE"))

lm(prop_p16_cells ~ AGE, data = p16_metadata) %>% summary()

#----- Test lm of senescence ~ each numeric variable + age.
numeric_vars <- p16_metadata %>% dplyr::select(where(is.numeric)) %>% dplyr::select_if(~ !any(is.na(.))) %>% names()
numeric_vars <- numeric_vars[!numeric_vars %in% c("SAMPLE", "AGE", "prop_p16_cells", "race", "expired_age")]
lmp <- function (modelobject) {
    # from https://stackoverflow.com/questions/5587676/pull-out-p-values-and-r-squared-from-a-linear-regression
    if (class(modelobject) != "lm") stop("Not an object of class 'lm' ")
    f <- summary(modelobject)$fstatistic
    p <- pf(f[1],f[2],f[3],lower.tail=F)
    attributes(p) <- NULL
    return(p)
}
for (numeric_var in numeric_vars) {
  # print(numeric_var)
  lm_model <- lm(as.formula(paste("prop_p16_cells ~", numeric_var, "+ AGE")), data = p16_metadata)
  lm_summary <- lm_model %>% summary() 
  # print(lm_summary)
  # lm_summary$coefficients["dementia_yn"]
  if ((lm_summary$coefficients[numeric_var,"Pr(>|t|)"] < 0.05) #| 
      # (lm_summary$coefficients["AGE","Pr(>|t|)"] < 0.05) | 
      # (lmp(lm_model) < 0.05)
      ) {
    message(paste(numeric_var, "has *significant* coefficients:"))
    print(lm_summary)
    print(lmp(lm_model))
  } else {
    message(paste(numeric_var, "has no significant coefficients."))
  }
  
}

#----- Group * Age Interaction in Explaining p16+ Proportion
lm(prop_p16_cells ~ GROUP + AGE, data = p16_metadata) %>% summary()
lm(prop_p16_cells ~ GROUP*AGE, data = p16_metadata) %>% summary()
for (any_var in names(p16_metadata)[!names(p16_metadata) %in% 
                                    c("SAMPLE", "CaseID", "BRAIN_REGION", "ID", 
                                      "SEQ_ID", "AD", "DLB", "VAD", "PSP", "HS",
                                      "DLDH", "MND", "CBD", "PICKS", "HD", "MSA",
                                      "FTLD-TDP", "TDP-43", "MS", "ACUTE_INFARCTS",
                                      "TangleP", "bf_nbm")]) { # don't have enough difference values for an interaction
  print(any_var)
  lm_model <- lm(as.formula(paste0("prop_p16_cells ~ `", any_var, "`*AGE")), data = p16_metadata)
  lm_summary <- lm_model %>% summary() 
  sig_age_interaction <- data.frame(lm_summary$coefficients[,"Pr(>|t|)"]) %>% 
    tibble::rownames_to_column("variable") %>% 
    dplyr::rename("p_val" = "lm_summary.coefficients....Pr...t....") %>%
    filter(endsWith(variable, ":AGE"),
           p_val < .05)
  if (nrow(sig_age_interaction) > 0) {
    message(paste(any_var, "has *significant* interaction coefficients:"))
    print(lm_summary)
    print(lmp(lm_model))
  } else {
    message(paste(any_var, "has no significant interaction coefficients."))
  }
}

# MuMIn::dredge(lm(prop_p16_cells ~ GROUP*AGE + AGE*DEMENTIA, 
#                  data = p16_metadata, 
#                  na.action = na.fail),
#               rank = 'AIC')

# ggformula::gf_point(prop_p16_cells ~ AGE, color = ~MCI, shape = ~MCI, data = p16_metadata) %>%
#   ggformula::gf_lm() + 
#   labs(title = "MFG: MCI*AGE", caption = "note, we don't have that many samples with MCI...") +
#   lims(y = c(0, .08))

In [ ]:
#---------- Seurat_Filtering
# Filter out low quality reads using selected thresholds 
# these will change with experiment
# seurat_filtered <- seurat_combined@meta.data %>% 
#                      filter(nUMI >= 500 & nUMI <= 20000)
# seurat_filtered <- subset(seurat_combined, cells = seurat_filtered$cells)
seurat_filtered <- subset(x = seurat_combined, 
                          subset = (nUMI >= nUMI_low) & 
                                   (nUMI <= nUMI_high) & 
                                   (nGene >= nGene_low) & 
                                   (nGene <= nGene_high) &
                                   (percent_mito < percent_mito_high) & 
                                   (percent_ribo < percent_ribo_high) &
                                   (log10GenesPerUMI > log10GenesPerUMI_low))

#---- save rds
saveRDS(seurat_filtered, file="seurat_results/seurat_filtered.rds")

In [ ]:
#---------- Seurat_Filtering_Senescence
#----- (repeating the same senescence demographic modeling as above, post-filter)

#----- Load data
seurat_filtered <- readRDS(file = "seurat_results/seurat_filtered.rds")

#----- Simple lm of senescence ~ age.
p16_metadata <- seurat_filtered@meta.data %>%
  mutate(CDKN2A_Positive = case_when(CDKN2A_Count > 0 ~ TRUE,
                                     CDKN2A_Count == 0 ~ FALSE)) %>%
  group_by(SAMPLE, CaseID) %>%
  summarize(total_cells = n(),
            total_p16_transcripts = sum(CDKN2A_Count),
            total_p16_cells = sum(CDKN2A_Positive),
            prop_p16_cells = total_p16_cells/total_cells) %>%
  left_join(y = covar, by = c("CaseID", "SAMPLE"))

lm(prop_p16_cells ~ AGE, data = p16_metadata) %>% summary()

#----- Test lm of senescence ~ each numeric variable + age.
numeric_vars <- p16_metadata %>% dplyr::select(where(is.numeric)) %>% dplyr::select_if(~ !any(is.na(.))) %>% names()
numeric_vars <- numeric_vars[!numeric_vars %in% c("SAMPLE", "AGE", "prop_p16_cells", "race", "expired_age")]
lmp <- function (modelobject) {
    # from https://stackoverflow.com/questions/5587676/pull-out-p-values-and-r-squared-from-a-linear-regression
    if (class(modelobject) != "lm") stop("Not an object of class 'lm' ")
    f <- summary(modelobject)$fstatistic
    p <- pf(f[1],f[2],f[3],lower.tail=F)
    attributes(p) <- NULL
    return(p)
}
for (numeric_var in numeric_vars) {
  # print(numeric_var)
  lm_model <- lm(as.formula(paste("prop_p16_cells ~", numeric_var, "+ AGE")), data = p16_metadata)
  lm_summary <- lm_model %>% summary() 
  # print(lm_summary)
  # lm_summary$coefficients["dementia_yn"]
  if ((lm_summary$coefficients[numeric_var,"Pr(>|t|)"] < 0.05) #| 
      # (lm_summary$coefficients["AGE","Pr(>|t|)"] < 0.05) |
      # (lmp(lm_model) < 0.05)
      ) {
    message(paste(numeric_var, "has *significant* coefficients:"))
    print(lm_summary)
    print(lmp(lm_model))
  } else {
    message(paste(numeric_var, "has no significant coefficients."))
  }
  
}

#----- Group * Age Interaction in Explaining p16+ Proportion
lm(prop_p16_cells ~ GROUP + AGE, data = p16_metadata) %>% summary()
lm(prop_p16_cells ~ GROUP*AGE, data = p16_metadata) %>% summary()
for (any_var in names(p16_metadata)[!names(p16_metadata) %in% 
                                    c("SAMPLE", "CaseID", "BRAIN_REGION", "ID", 
                                      "SEQ_ID", "AD", "DLB", "VAD", "PSP", "HS",
                                      "DLDH", "MND", "CBD", "PICKS", "HD", "MSA",
                                      "FTLD-TDP", "TDP-43", "MS", "ACUTE_INFARCTS",
                                      "TangleP", "bf_nbm")]) { # don't have enough difference values for an interaction
  print(any_var)
  lm_model <- lm(as.formula(paste0("prop_p16_cells ~ `", any_var, "`*AGE")), data = p16_metadata)
  lm_summary <- lm_model %>% summary() 
  sig_age_interaction <- data.frame(lm_summary$coefficients[,"Pr(>|t|)"]) %>% 
    tibble::rownames_to_column("variable") %>% 
    dplyr::rename("p_val" = "lm_summary.coefficients....Pr...t....") %>%
    filter(endsWith(variable, ":AGE"),
           p_val < .05)
  if (nrow(sig_age_interaction) > 0) {
    message(paste(any_var, "has *significant* interaction coefficients:"))
    print(lm_summary)
    print(lmp(lm_model))
  } else {
    message(paste(any_var, "has no significant interaction coefficients."))
  }
}

# MuMIn::dredge(lm(prop_p16_cells ~ GROUP*AGE + AGE*DEMENTIA, 
#                  data = p16_metadata, 
#                  na.action = na.fail),
#               rank = 'AIC')

ggplot(p16_metadata, aes(y = prop_p16_cells, x = AGE, color = dementia_nos, shape = dementia_nos)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  lims(y = c(0, NA)) +
  labs(title = "MFG (post-filter): Dementia*Age")

ggplot(p16_metadata, aes(y = prop_p16_cells, x = AGE, color = MCI, shape = MCI)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  lims(y = c(0, NA)) +
  labs(title = "MFG (post-filter): MCI*AGE")

p16_metadata %>%
  mutate(bf_trans = as.character(bf_trans)) %>%
  ggplot(aes(y = prop_p16_cells, x = AGE, color = bf_trans)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  lims(y = c(0, NA)) +
  labs(title = "MFG (post-filter): bf_trans*AGE")

p16_metadata %>%
  # mutate(bf_trans = as.character(bf_trans)) %>%
  ggplot(aes(y = prop_p16_cells, x = AGE, color = bf_trans)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  lims(y = c(0, NA)) +
  labs(title = "MFG (post-filter): bf_trans*AGE") +
  facet_wrap(~bf_trans)

p16_metadata %>%
  # mutate(bf_trans = as.character(bf_trans)) %>%
  ggplot(aes(y = prop_p16_cells, x = AGE, color = bf_cing)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  lims(y = c(0, NA)) +
  labs(title = "MFG (post-filter): bf_cing*AGE") +
  facet_wrap(~bf_cing)


p16_metadata %>%
  # mutate(bf_trans = as.character(bf_trans)) %>%
  ggplot(aes(y = prop_p16_cells, x = AGE, color = `Braak score`)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE) +
  lims(y = c(0, NA)) +
  labs(title = "MFG (post-filter): Braak score*AGE") +
  facet_wrap(~`Braak score`)

p16_metadata %>%
  # mutate(bf_trans = as.character(bf_trans)) %>%
  ggplot(aes(y = prop_p16_cells, x = AGE)) +
  geom_point() +
  geom_smooth(method = "lm", se = TRUE) +
  lims(y = c(0, NA)) +
  labs(title = "MFG (post-filter): AGE") 

In [ ]:
#---------- Cell_Cycle_Scoring
#----- Load cell cycle markers
load(file="seurat_results/seurat_cycle.rda")

#----- Identify Cell cycle phase for each cell
seurat_cellcycle <- function(seurat_obj, number_features) {
  #----- Normalize the counts
  phase <- NormalizeData(seurat_obj)
  #----- Identify the most variable genes
  phase <- FindVariableFeatures(object = phase, 
                                selection.method = "vst",
                                nfeatures = number_features, 
                                verbose = FALSE)
  #----- Scale the counts
  phase <- ScaleData(object = phase)
  #----- Perform PCA
  phase <- RunPCA(object = phase,
                  verbose = FALSE)
  #----- Score cells for cell cycle
  phase <- CellCycleScoring(object = phase, 
                            g2m.features = g2m_genes, 
                            s.features = s_genes)
  return(phase)
} 
seurat_phase <- seurat_cellcycle(seurat_obj = seurat_filtered, 
                                 number_features = 2000)
  
#---------- Data
#----- Save Data
saveRDS(seurat_phase, file="seurat_results/seurat_phase.rds")


# PROCESSING

> SOURCE: [Harmony-RNA-Workflow/scripts/main/process.R](https://github.com/shahrozeabbas/Harmony-RNA-Workflow/blob/main/scripts/main/process.R) 

The main processing requires subsetting to the QC features, followed by normalization.  From there identifying highly variable genes to scale, score (e.g. cell cycle).  Batch correction ought to be considered as part of scaling.


In [ ]:
# TEAM LEE

library(Seurat)
library(SeuratObject)

#----- Load data
seurat_filtered <- readRDS(file = "seurat_results/seurat_filtered.rds")

#---------- Seurat_Batch_Correction_RPCA
# Ensure the dataset uses the RNA assay
DefaultAssay(seurat_filtered) <- "RNA"

#----- Split Data 
# Split data into individual ‘assays’.
seurat_split <- SplitObject(object = seurat_filtered, 
                            split.by = "SAMPLE")

#----- Normalize and Identify Variable Features
# LogNormalize: Feature counts for each cell are divided by the  
# total counts for that cell and multiplied by the scale.factor  
# (10,000 by default), and log-transforms the result. 
# Normalized values are stored in seurat_obj[["RNA"]]@data.
seurat_split <- lapply(X = seurat_split, FUN = function(x){
    x <- NormalizeData(object = x)
    x <- FindVariableFeatures(object = x, 
                              selection.method = "vst", 
                              nfeatures = 2000)
})

#----- Select features repeated across datasets for Integration
features <- SelectIntegrationFeatures(object.list = seurat_split)

#---------- Cell_Cycle_Scoring
#----- Load cell cycle markers
load(file="seurat_results/seurat_cycle.rda")
seurat_split <- lapply(X = seurat_split, FUN = function(x) {
    x <- ScaleData(object = x, 
                   features = features, 
                   vars.to.regress = "percent_mito",
                   verbose = FALSE)
    x <- RunPCA(object = x, 
                features = features, 
                verbose = FALSE)
    x <- CellCycleScoring(object = x, 
                          g2m.features = g2m_genes, 
                          s.features = s_genes)
})

#----- Perform integration
seurat_anchors <- FindIntegrationAnchors(object.list = seurat_split, 
                                         anchor.features = features, 
                                         reduction = "rpca")
rm(seurat_split)

#---- Integrate across samples
# this command creates an 'integrated' data assay
seurat_rpca <- IntegrateData(anchorset = seurat_anchors)
rm(seurat_anchors)

# specify that we will perform downstream analysis on the corrected data
# original unmodified data still resides in the 'RNA' assay
DefaultAssay(seurat_rpca) <- "integrated"

# Run the standard workflow for visualization and clustering
seurat_rpca <- ScaleData(seurat_rpca, 
                         vars.to.regress = "percent_mito",
                         verbose = FALSE)

#---- Run PCA and UMAP
seurat_rpca <- RunPCA(object = seurat_rpca, 
                      verbose = FALSE)
# ElbowPlot(object = seurat_rpca, 
#           ndims = 50)
seurat_rpca <- RunUMAP(seurat_rpca, 
                       reduction = "pca", 
                       dims = 1:30)

#---- save rds
saveRDS(seurat_rpca, file="seurat_results/seurat_rpca.rds")



In [ ]:
#---------- Seurat_Batch_Correction_Harmony
#----- Load data
seurat_rpca <- readRDS(file = "seurat_results/seurat_rpca.rds")

#---------- PCA
#----- Elbow Plot
plot <- ElbowPlot(object = seurat_rpca, 
                  ndims = 50) + 
          theme(plot.title = element_text(size = 20,
                                          hjust = 0.5, 
                                          face = "bold")) +
          ggtitle("PCA Elbow Plot")

plot
png(file="seurat_results/figures/Seurat_Batch_Correction_RPCA_PCA_Elbowplot.png", 
    width=600, 
    height=400)
plot
dev.off()
#----- PCA Group
plot <- PCAPlot(object = seurat_rpca,
                group.by = "SAMPLE",
                split.by = "GROUP")  +
          ggtitle("PCA") + 
          scale_color_manual(values = color_sample)
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_RPCA_PCA_group.png", 
    width=800, 
    height=400)
plot
dev.off()

#----- Explore heatmap of PCs
# Printing out the most variable genes driving PCs
# print(x = seurat_rpca[["pca"]], dims = 1:9, nfeatures = 5)
# -- doesn't return a ggplot object, so we'll run twice...
# -- one for RStudio viewing, one to save as png
DimHeatmap(object = seurat_rpca, 
           dims = 1:9, 
           cells = 500, 
           balanced = TRUE)
png(file="seurat_results/figures/Seurat_Batch_Correction_RPCA_PCA_heatmap.png", 
    width=1500, 
    height=1500,
    pointsize = 16)
DimHeatmap(object = seurat_rpca, 
           dims = 1:9, 
           cells = 500, 
           balanced = TRUE)
dev.off()

#---------- UMAP                             
plot <- DimPlot(object = seurat_rpca, 
                reduction = "umap",
                shuffle = TRUE,
                group.by = "GROUP") + 
          scale_color_manual(values = color_group)
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_RPCA_UMAP_group.png", 
    width=600, 
    height=600)
plot
dev.off()
plot <- DimPlot(object = seurat_rpca, 
                reduction = "umap",
                shuffle = TRUE,
                split.by = "GROUP") + 
          scale_color_manual(values = color_sample)
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_RPCA_UMAP_sample.png", 
    width=1000, 
    height=600)
plot
dev.off()
plot <- DimPlot(object = seurat_rpca, 
                reduction = "umap",
                shuffle = TRUE,
                split.by = "GROUP") + 
          scale_color_manual(values = color_sample) +
          theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_RPCA_UMAP_sample_no_legend.png", 
    width=1000, 
    height=600)
plot
dev.off()
plot <- DimPlot(object = seurat_rpca, 
                reduction = "umap",
                shuffle = TRUE,
                split.by = "SAMPLE",
                ncol = 5) + 
          scale_color_manual(values = color_sample) +
          theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_RPCA_UMAP_sample_all.png", 
    width=2000, 
    height=1000)
plot
dev.off()



rm(seurat_rpca)

# HARMONIZE

> SOURCE: [Harmony-RNA-Workflow/scripts/main/harmony.R](https://github.com/shahrozeabbas/Harmony-RNA-Workflow/blob/main/scripts/main/harmony.R) 

This is the final step towards our endpoint.  Dimension reduction, and cell-type identification will be performed on the 


Harmony for batch integration when using large samples that cause memory issues  

In [ ]:
#---------- Seurat_Batch_Correction
# The data is preprocessed by splitting into individual ‘assays’. 
# In this experiment, each individual sample is considered an ‘assay’.
# Ensure the dataset used here is the non-normalized dataset
DefaultAssay(seurat_filtered) <- "RNA"
#----- Load cell cycle markers
load(file="seurat_results/seurat_cycle.rda")

#---------- Harmony
#----- Batch Correction
seurat_batch <- function(seurat_obj, number_features, sample, dimensions) {
  #----- Normalize the counts
  harmony <- NormalizeData(seurat_obj)
  #----- Identify the most variable genes
  harmony <- FindVariableFeatures(object = harmony, 
                                  selection.method = "vst",
                                  nfeatures = number_features, 
                                  verbose = FALSE)
  #----- Scale the counts
  harmony <- ScaleData(object = harmony)
  #----- Perform PCA
  harmony <- RunPCA(object = harmony,
                    verbose = FALSE)
  #----- Score cells for cell cycle
  harmony <- CellCycleScoring(object = harmony, 
                              g2m.features = g2m_genes, 
                              s.features = s_genes)
  #---- Harmony Integration
  harmony <- RunHarmony(object = harmony, 
                        group.by.vars = sample)
  #---- Perform UMAP
  harmony <- RunUMAP(object = harmony, 
                     reduction = "harmony", 
                     dims = dimensions)
  return(harmony)
} 
seurat_harmony <- seurat_batch(seurat_obj = seurat_filtered, 
                               number_features = 2000,
                               sample = "SAMPLE",
                               dimensions = 1:30)  
# ElbowPlot(object = seurat_harmony, ndims = 50)
# DimPlot(seurat_harmony, split.by = "sample", ncol = 6) + NoLegend()

#---------- Data
#----- Save Data
saveRDS(seurat_harmony, file="seurat_results/seurat_harmony.rds")
#----- Remove Data
rm(seurat_filtered)

In [ ]:
#---------- Seurat_Batch_Correction_Harmony
#----- Load data
seurat_harmony <- readRDS(file = "seurat_results/seurat_harmony.rds")

#---------- PCA
#----- Elbow Plot
plot <- ElbowPlot(object = seurat_harmony, 
                  ndims = 50) + 
          theme(plot.title = element_text(size = 20,
                                          hjust = 0.5, 
                                          face = "bold")) +
          ggtitle("PCA Elbow Plot")
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_PCA_Elbowplot.png", 
    width=600, 
    height=400)
plot
dev.off()
#----- PCA Group
plot <- PCAPlot(object = seurat_harmony,
                group.by = "SAMPLE",
                split.by = "GROUP") +
          ggtitle("PCA") + 
          scale_color_manual(values = color_sample)
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_PCA_group.png", 
    width=800, 
    height=400)
plot
dev.off()
plot <- PCAPlot(object = seurat_harmony,
                group.by = "SAMPLE",
                split.by = "GROUP") +
          ggtitle("PCA") + 
          scale_color_manual(values = color_sample) +
          theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_PCA_group_no_legend.png", 
    width=800, 
    height=400)
plot
dev.off()

#----- Explore heatmap of PCs
# Printing out the most variable genes driving PCs
# print(x = seurat_harmony[["pca"]], dims = 1:9, nfeatures = 5)
# -- doesn't return a ggplot object, so we'll run twice...
# -- one for RStudio viewing, one to save as png
DimHeatmap(object = seurat_harmony, 
           dims = 1:9, 
           cells = 500, 
           balanced = TRUE)
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_PCA_heatmap.png", 
    width=1500, 
    height=1500,
    pointsize = 16)
DimHeatmap(object = seurat_harmony, 
           dims = 1:9, 
           cells = 500, 
           balanced = TRUE)
dev.off()

#---------- UMAP 
plot <- DimPlot(object = seurat_harmony, 
                reduction = "umap",
                shuffle = TRUE,
                group.by = "GROUP") + 
          scale_color_manual(values = color_group)
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_UMAP_group.png", 
    width=600, 
    height=600)
plot
dev.off()
plot <- DimPlot(object = seurat_harmony, 
                reduction = "umap",
                shuffle = TRUE,
                split.by = "GROUP") + 
          scale_color_manual(values = color_sample) 
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_UMAP_sample.png", 
    width=1000, 
    height=600)
plot
dev.off()
plot <- DimPlot(object = seurat_harmony, 
                reduction = "umap",
                shuffle = TRUE,
                split.by = "GROUP") + 
          scale_color_manual(values = color_sample) +
          theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_UMAP_sample_no_legend.png", 
    width=1000, 
    height=600)
plot
dev.off()
plot <- DimPlot(object = seurat_harmony, 
                reduction = "umap",
                shuffle = TRUE,
                split.by = "SAMPLE",
                ncol = 5) + 
          scale_color_manual(values = color_sample) +
          theme(legend.position = "none")
plot
png(file="seurat_results/figures/Seurat_Batch_Correction_Harmony_UMAP_sample_all.png", 
    width=2000, 
    height=1000)
plot
dev.off()


## Seurat Clustering

Seurat uses a graph-based clustering approach, which embeds cells in a graph  
structure, using a K-nearest neighbor (KNN) graph (by default), with edges  
drawn between cells with similar gene expression patterns.  

The resolution is an important argument that sets the “granularity” of the  
downstream clustering and will need to be optimized for every individual  
experiment. For datasets of 3,000 - 5,000 cells, the resolution set between   
0.4-1.4 generally yields good clustering. Increased resolution values lead 
to a greater number of clusters, which is often required for larger datasets  
  
* Resolution 0.6  

In [ ]:
#---------- Seurat_Clustering
seurat_knn <- function(seurat_obj, reduction_assay, dimensions, resolutions) {
  # Determine the K-nearest neighbor graph
  cluster <- FindNeighbors(object = seurat_obj, 
                           reduction = reduction_assay, 
                           dims = dimensions)
  # Clusters for various resolutions
  cluster <- FindClusters(object = cluster,
                          resolution = resolutions)
  return(cluster)
} 
seurat_cluster <- seurat_knn(seurat_obj = seurat_harmony, 
                             reduction_assay = "harmony",
                             dimensions = 1:30,
                             resolutions = seq(0.2,1.2,0.2))

#---------- Seurat_Clustering Resolution
#---------- Seurat_Clustering_UMAP
#----- Assign identity of clusters
Idents(object = seurat_cluster) <- "RNA_snn_res.0.6"
#----- Save Resolution 
seurat_cluster@meta.data$seurat_clusters <- Idents(seurat_cluster)
#----- Plot UMAP                             
DimPlot(object = seurat_cluster, 
        reduction = "umap", 
        label = TRUE) + 
          NoLegend()

#---------- Data
#----- Save Data
saveRDS(seurat_cluster, file="seurat_results/seurat_cluster.rds")
#----- Remove Data
rm(seurat_harmony)

In [ ]:
#---------- Seurat_Clustering_Figures
#----- Load data
seurat_cluster <- readRDS(file = "seurat_results/seurat_cluster.rds")
#----- Plot UMAP                             
plot <- DimPlot(object = seurat_cluster, 
                reduction = "umap", 
                label = TRUE) + 
          NoLegend()
plot
png(file="seurat_results/figures/Seurat_Clustering_UMAP_resolution.png", 
    width=600, 
    height=600)
plot
dev.off()
#----- Violin Plot
plot <- VlnPlot(object = seurat_cluster, 
                features = c("nGene", "nUMI", "log10GenesPerUMI", "percent_mito"), #"mitoRatio"), 
                ncol = 1, 
                pt.size=0, 
                group.by="seurat_clusters")
plot
png(file="seurat_results/figures/Seurat_Clustering_violin.png", 
    width=600, 
    height=800)
plot
dev.off()
#----- Feature Plot
plot <- cowplot::plot_grid(nrow=2, ncol=2,
  FeaturePlot(seurat_cluster, 
              reduction = "umap", 
              features = "nGene", 
              pt.size = 0.1, 
              order = TRUE,
              label = TRUE) + 
    scale_colour_gradientn(colours = viridis(n = 11)),
  FeaturePlot(seurat_cluster, 
              reduction = "umap", 
              features = "nUMI",
              pt.size = 0.1, 
              order = TRUE,
              label = TRUE) + 
    scale_colour_gradientn(colours = viridis(n = 11)),
  FeaturePlot(seurat_cluster, 
              reduction = "umap", 
              features = "log10GenesPerUMI", 
              pt.size = 0.1, 
              order = TRUE,
              label = TRUE) + 
    scale_colour_gradientn(colours = viridis(n = 11)),
  FeaturePlot(seurat_cluster, 
              reduction = "umap", 
              features = "percent_mito", #"mitoRatio",
              pt.size = 0.1, 
              order = TRUE,
              label = TRUE) + 
    scale_colour_gradientn(colours = viridis(n = 11)))
plot
png(file="seurat_results/figures/Seurat_Clustering_feature.png", 
    width=800, 
    height=600)
plot
dev.off()

#---------- Seurat Clustering Proportions
#----- Clustering Proportions
cluster_proportions <- seurat_cluster@meta.data %>% 
  dplyr::group_by(GROUP, SAMPLE) %>% 
  dplyr::count(seurat_clusters) %>% 
  dplyr::rename(Count = n) %>%
  dplyr::mutate(Proportion = Count / sum(Count)) %>% 
  dplyr::select(-Count) %>%
  tidyr::spread(seurat_clusters, Proportion)
#----- Save Results
write_delim(cluster_proportions,
            paste0("seurat_results/", NameProject, "_Clustering_Proportions.txt"),
            delim = "\t")
#----- Cell Cycle Proportions Figure Group
plot <- ggGroup + 
  geom_boxplot(data = seurat_cluster@meta.data %>% 
                        dplyr::group_by(GROUP, SAMPLE) %>% 
                        dplyr::count(seurat_clusters) %>% 
                        dplyr::rename(Count = n) %>%
                        dplyr::mutate(Proportion = Count / sum(Count)),
               aes(x = GROUP, y = Proportion, fill = GROUP)) + 
  ggtitle("Number of Cells per Cluster") + 
  theme(legend.position = "none") + 
  facet_wrap(~seurat_clusters, scale="free", ncol = 5)
plot
png(file="seurat_results/figures/Seurat_Clustering_Proportion_group.png", 
    width=800, 
    height=600)
plot
dev.off()

#----- Cell Cycle Proportions Data
seurat_cluster@meta.data %>% 
  dplyr::group_by(GROUP, SAMPLE) %>% 
  dplyr::count(seurat_clusters) %>% 
  dplyr::rename(Count = n) %>%
  dplyr::mutate(Proportion = Count / sum(Count)) %>%
  kable(caption = "Number of Cells per Cluster", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))

seurat_cluster@meta.data %>% 
  dplyr::group_by(BRAIN_REGION, SAMPLE) %>% 
  dplyr::count(seurat_clusters) %>% 
  dplyr::rename(Count = n) %>%
  dplyr::mutate(Proportion = Count / sum(Count)) %>%
  kable(caption = "Number of Cells per Cluster", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))


In [ ]:
#---------- Seurat_Clustering_Phase
#----- Violin Plot
plot <- VlnPlot(object = seurat_cluster, 
                features = c("S.Score", "G2M.Score"), 
                ncol = 1, 
                pt.size=0, 
                group.by="seurat_clusters")
plot
png(file="seurat_results/figures/Seurat_Clustering_Phase_violin.png", 
    width=600, 
    height=400)
plot
dev.off()
#----- Feature Plot
plot <- cowplot::plot_grid(nrow=1, ncol=2,
  FeaturePlot(seurat_cluster, 
              reduction = "umap", 
              features = "S.Score", 
              pt.size = 0.1, 
              order = TRUE,
              label = TRUE) + 
    scale_colour_gradientn(colours = viridis(n = 11)),
  FeaturePlot(seurat_cluster, 
              reduction = "umap", 
              features = "G2M.Score",
              pt.size = 0.1, 
              order = TRUE,
              label = TRUE) + 
    scale_colour_gradientn(colours = viridis(n = 11)))
plot
png(file="seurat_results/figures/Seurat_Clustering_Phase_feature.png", 
    width=800, 
    height=300)
plot
dev.off()

## Seurat Annotation

Databases for cell-type markers  
[PanglaoDB](https://panglaodb.se/markers.html?cell_type=%27Astrocytes%27)  
[CellMarker](http://bio-bigdata.hrbmu.edu.cn/CellMarker/search.jsp?species=Human&tissue=Brain&cellname=Astrocyte)  

In [ ]:
#---------- Seurat_Annotation
#----- Load data
seurat_cluster <- readRDS(file = "seurat_results/seurat_cluster.rds")
#----- Stacked Violin Plots
vlnplot_gg <- function(seurat_obj, 
                       feature, 
                       pt_size = 0, 
                       plot_margin = unit(c(-0.75, 0, -0.75, 0), "cm"),
                       ...){
  p <- VlnPlot(seurat_obj, 
               features = feature, 
               pt.size = pt_size, 
               ...) + 
         ggtitle("") + 
         xlab("") + 
         ylab(feature) + 
         theme(legend.position = "none", 
               axis.text.x = element_blank(), 
               axis.ticks.x = element_blank(), 
               axis.title.y = element_text(size = rel(1), angle = 0), 
               axis.text.y = element_text(size = rel(1)), 
               plot.margin = plot_margin) 
  return(p)
}
vlnplot_max<- function(p){
  ymax <- max(ggplot_build(p)$layout$panel_scales_y[[1]]$range$range)
  return(ceiling(ymax))
}
vlnplot_stacked <- function(seurat_obj, 
                            features,
                            pt_size = 0, 
                            plot_margin = unit(c(-0.75, 0, -0.75, 0), "cm"),
                            ...){
  plot_list <- purrr::map(features, function(feature) 
    vlnplot_gg(seurat_obj = seurat_obj,
               feature = feature, 
               ...))
  # Add back x-axis title to bottom plot. 
  plot_list[[length(plot_list)]] <- plot_list[[length(plot_list)]] + 
                                      theme(axis.text.x = element_text(), 
                                            axis.ticks.x = element_line())
  # Change the y-axis tick to only max value 
  ymaxs <- purrr::map_dbl(plot_list, vlnplot_max)
  plot_list <- purrr::map2(plot_list, ymaxs, function(x,y) x + 
                             scale_y_continuous(breaks = c(y)) + 
                             expand_limits(y = y))
  p <- patchwork::wrap_plots(plotlist = plot_list, ncol = 1)
  return(p)
}

#---------- Cell-type Marker Expression
# rownames(seurat_markers@assays$RNA)[grepl(pattern = "CD3", rownames(seurat_markers@assays$RNA))]
#----- UMAP
DimPlot(seurat_cluster, 
        reduction="umap", 
        label = TRUE) + NoLegend()

#---------- Astrocyte_Cells
# GFAP, AQP4 (2)
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("AQP4", "GFAP"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("AQP4", "GFAP"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Astrocyte_Restricted_Precursors_Cells
# CD44 () -- interesting sub cluster by 2/15
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("CD44"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("CD44"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- B_Cells
# CD27, CD79A not detected 
# CD38, CD24 ()
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("CD27", "CD38", "CD24"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("CD27", "CD38", "CD24"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- CADPS2 High
# CADPS2, TIAM1 (?? kind of everywhere, but not both everywhere...)
# NR4A2 for DA neurons
# "MAP2", "SCN2A" for neurons
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("CADPS2", "TIAM1", "NR4A2", "MAP2", "SCN2A"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("CADPS2", "TIAM1", "NR4A2", "MAP2", "SCN2A"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Choroid_Plexus_Cells
# KL, CHMP1A, TTR, SLC26A11 ()
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("KL", "CHMP1A", "SLC26A11"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("KL", "CHMP1A", "SLC26A11"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Dopaminergic_Neurons
# # TH ()
# vlnplot_stacked(seurat_obj = seurat_cluster,
#                 features = c("TH"))
# FeaturePlot(object = seurat_cluster,
#             reduction = "umap",
#             features = c("TH"),
#             min.cutoff = "q10",
#             max.cutoff = "q90",
#             order = TRUE,
#             label = TRUE)

# DA scType
# "TH", "SLC6A3", "FOXA2", "KCNJ6", "NR4A2", "LMX1B", "DBH", "SLC6A2", "PPP1R1B"
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("KCNJ6", "NR4A2", "DBH", "PPP1R1B", "CADPS2"))#, "TH"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("KCNJ6", "NR4A2", "DBH", "PPP1R1B", "CADPS2"),# "TH"),
            min.cutoff = "q10",
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Dopaminergic_Neurons_PD_Resistant_Macosko
# https://www.nature.com/articles/s41593-022-01061-1.pdf
# TH, CALB1, TMEM200A () -- TH not a slot in the data
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("CALB1", "TMEM200A"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("CALB1", "TMEM200A"),
            min.cutoff = "q10",
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)


#---------- Dopaminergic_Neurons_PD_Vulnerable_Macosko
# https://www.nature.com/articles/s41593-022-01061-1.pdf
# TH, AGTR1, SOX6() -- TH and AGTR1 not a slot in the data
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("SOX6"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("SOX6"),
            min.cutoff = "q10",
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Endothelial_Cells
# FLT1, DUSP1, RGS5, CLDN5 (9)
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("FLT1", "DUSP1", "RGS5", "CLDN5"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("FLT1", "DUSP1", "RGS5", "CLDN5"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Ependymal_Cells
# FOXJ1 ()
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("FOXJ1"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("FOXJ1"),
            min.cutoff = "q10",
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Excitatory_Neurons (Glutaminergic neurons)
# I'm adding CBLN2? (ask about this - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5038589/)
# SATB2, SLC17A6, SLC17A7 (1,3,4,5?,11,16,18,19,21,22,23) #8,13?, (if 12, 21... 24. (I think 28,29 don't really fit) (12?)
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("SATB2", "SLC17A6", "SLC17A7", "CBLN2"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("SATB2", "SLC17A6", "SLC17A7", "CBLN2"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Inhibitory_Neurons_LHX6_Positive (GABAergic neurons)
# https://www.nature.com/articles/s41422-018-0053-3
# GAD1, GAD2, GRIK1: + LHX6, NXPH1, PAM (7, 15, 26)
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("GAD2", "GAD1", "GRIK1", "LHX6", "NXPH1", "PAM"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("GAD2", "GAD1", "GRIK1", "LHX6", "NXPH1", "PAM"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

# #---------- Inhibitory_Neurons_CALB2_Positive (GABAergic neurons)
# https://www.nature.com/articles/s41422-018-0053-3
# GAD1, GAD2, GRIK1: + CALB2 (+"NR2F2", "CAMLG"??) (6)
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("GAD2", "GAD1", "GRIK1", "CALB2"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("GAD2", "GAD1", "GRIK1", "CALB2"),
            min.cutoff = "q10",
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Inhibitory_Neurons_Other (remaining) (GABAergic neurons)
# GAD1, GAD2, GRIK1 (14, 7, 17, 25, 6, [ont really 26&27][part of 5?])
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("GAD2", "GAD1", "GRIK1"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("GAD2", "GAD1", "GRIK1"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Macrophage_Cells
# CD68 TREM2 () -- part of 12, but microglial fits better here
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("CD68", "TREM2"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("CD68", "TREM2"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Microglial_Cells
# PLXDC2, CSF3R, 
# APBB1IP, CSF1R, CD74, (12,20?)
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("APBB1IP", "CSF1R", "CD74")) 
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("APBB1IP", "CSF1R", "CD74"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Monocyte_Cells
# CD40 CXCR4 ()
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("CD40", "CXCR4"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("CD40", "CXCR4"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Neural_Stem_Cells
# SOX2 SOX9 (2? -- no, astrocytes make more sense here.)
vlnplot_stacked(seurat_obj = seurat_cluster, 
                features = c("SOX2", "SOX9"))
FeaturePlot(object = seurat_cluster, 
            reduction = "umap",
            features = c("SOX2", "SOX9"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Oligodendrocyte_Cells
# MOG, MOBP (0) (doublet? = 15, half of 10&12, 27?,28?)
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("MOG", "MOBP")) 
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("MOG", "MOBP"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Oligodendrocyte_Precursor_Cells
# OLIG1, VCAN (10)
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("VCAN", "OLIG1"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("VCAN", "OLIG1"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- Pericyte_Cells
# PDGFRB (maybe half of 9, and some in 2)
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("PDGFRB")) #, "CSPG4", "ACTA2", "NES"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("PDGFRB"), #, "CSPG4", "ACTA2", "NES"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

#---------- T_Cells
# CCR6 CD4 ()
vlnplot_stacked(seurat_obj = seurat_cluster,
                features = c("CCR6", "CD4"))
FeaturePlot(object = seurat_cluster,
            reduction = "umap",
            features = c("CCR6", "CD4"), 
            min.cutoff = "q10", 
            max.cutoff = "q90",
            order = TRUE,
            label = TRUE)

In [ ]:
#---------- Seurat_Annotation_Assign
#----- Annotate each cluster
seurat_anno <- RenameIdents(object = seurat_cluster, 
                            `0` =  "Oligodendrocyte_Cells",
                            `1` =  "Excitatory_Neurons",
                            `2` =  "Astrocyte_Cells",
                            `3` =  "Excitatory_Neurons",
                            `4` =  "Excitatory_Neurons",
                            `5` =  "Excitatory_Neurons", #??
                            `6` =  "Inhibitory_Neurons",
                            `7` =  "Inhibitory_Neurons",
                            `8` =  "Excitatory_Neurons",
                            `9` =  "Endothelial_Cells",
                            `10` = "Oligodendrocyte_Precursor_Cells", #?Oligodendrocyte_Cells (these are transitioning!!!?
                            `11` = "Excitatory_Neurons",
                            `12` = "Microglial_Cells",
                            `13` = "Excitatory_Neurons",
                            `14` = "Inhibitory_Neurons",
                            `15` = "doublet_astro_oligo_1", #?Oligodendrocyte_Cells
                            `16` = "Excitatory_Neurons",
                            `17` = "Inhibitory_Neurons", 
                            `18` = "Excitatory_Neurons",
                            `19` = "Excitatory_Neurons",
                            `20` = "Microglial_Cells", #??? Microglial_Cells
                            `21` = "Excitatory_Neurons",
                            `22` = "Excitatory_Neurons",
                            `23` = "Excitatory_Neurons",
                            `24` = "doublet_astro_oligo_2",
                            `25` = "Inhibitory_Neurons",
                            `26` = "Inhibitory_Neurons", #??
                            `27` = "doublet_InN_oligo", #??
                            `28` = "doublet_ExN_oligo", #Excitatory_Neurons???
                            `29` = "Excitatory_Neurons") #Excitatory_Neurons
                            
#----- save these annotations in meta.data
seurat_anno@meta.data$cluster_annotation <- as.factor(as.character(Idents(seurat_anno)))
#----- UMAP
DimPlot(seurat_anno, 
        reduction="umap", repel = TRUE,
        label = TRUE) + NoLegend()

#---------- Seurat_Annotation_Manual
#----- Metadata
metadata <- seurat_anno@meta.data
# metadata$cluster_annotation <- "Na"
# #----- Manual Edit
# anno_umap <- DimPlot(seurat_anno, reduction="umap", label = TRUE) 
# anno_cells <- CellSelector(plot = anno_umap)
# metadata$cluster_annotation[metadata$cells %in% anno_cells] <- "Oligodendrocyte_Precursor_Cells"
#----- Metadata
seurat_anno@meta.data <- metadata
#----- Assign identity of clusters
Idents(object = seurat_anno) <- "cluster_annotation"
#----- UMAP
DimPlot(seurat_anno, 
        reduction="umap", repel = TRUE,
        label = TRUE) + NoLegend()

#---------- Data
#----- Save Data
saveRDS(seurat_anno, file="seurat_results/seurat_anno.rds")
#----- Remove Data
rm(seurat_cluster, metadata, anno_umap, anno_cells)


In [ ]:
#---------- Seurat_Annotation_Figures
#----- Load data
seurat_anno <- readRDS(file = "seurat_results/seurat_anno.rds")
#----- Plot UMAP                             
plot <- (DimPlot(object = seurat_anno, 
                 reduction = "umap" #,
                 # custom colors: not used. 
                 # cols = c("#2887C4", # oligodendrocytes
                 #          "#ACA4E2", # exN c1
                 #          "#3FA294", # astrocytes
                 #          "#92ABE1", # exN c3
                 #          "#FBCC6B", # inhibitory neurons
                 #          "#59B7D2", # exN c4
                 #          "#1B3C74", # OPCs
                 #          "#76B2DB", # exN c5
                 #          "#71299A", # microglial
                 #          "#40BBC5", # exN c2
                 #          "#A9C47F", # endothelial
                 #          "#38BDB5", # exN c6
                 #          "#45BEA4", # exN c8
                 #          "#5CBD92"  # exN c7
                 #          )
                 ) + 
          NoLegend()) %>%
          LabelClusters(plot = ., id = "ident", repel = TRUE, box.padding = 1)
plot
ggsave(filename = "seurat_results/figures/Seurat_Annotation_UMAP_resolution_high_VAI.png", 
       plot = plot,
       width = 600*6,
       height = 600*6,
       dpi = 500,
       units = "px")
png(file="seurat_results/figures/Seurat_Annotation_UMAP_resolution_VAI.png", 
    width=600, 
    height=600)
plot
dev.off()
#----- Plot UMAP group
plot <- (DimPlot(object = seurat_anno, 
                reduction = "umap", 
                split.by = "GROUP") + 
          NoLegend()) %>%
          LabelClusters(plot = ., id = "ident", repel = TRUE, box.padding = 1.5)
plot
png(file="seurat_results/figures/Seurat_Annotation_UMAP_resolution_group.png", 
    width=1000, 
    height=600)
plot
dev.off()
#----- Plot UMAP group (no labels)
plot <- (DimPlot(object = seurat_anno, 
                reduction = "umap", 
                split.by = "GROUP") + 
          NoLegend())
plot
png(file="seurat_results/figures/Seurat_Annotation_UMAP_resolution_group_no_labels.png", 
    width=1000, 
    height=600)
plot
dev.off()
#----- Cell Markers
# levels_active_ident <- factor(sort(levels(seurat_anno@active.ident)))
# seurat_anno@active.ident <- levels_active_ident
plot <- DotPlot(object = seurat_anno,
                features = c("AQP4",    # Astrocyte_Cells
                             "FLT1",    # Endothelial_Cells
                             "SLC17A7", # Excitatory_Neurons
                             # "CALB2",   # Inhibitory_Neurons (CALB2+)
                             # "LHX6",    # Inhibitory_Neurons (LHX6+)
                             "GAD1",    # Inhibitory_Neurons (all)
                             "APBB1IP", # Microglial_Cells
                             "MOBP",    # Oligodendrocyte_Cells
                             "VCAN"),   # Oligodendrocyte_Precursor_Cells
                cols = c("blue", "red"),  
                dot.scale = 8) + 
          RotatedAxis()
plot
png(file="seurat_results/figures/Seurat_Annotation_Cell_Markers.png", 
    width=800, 
    height=400)
plot
dev.off()

In [ ]:
#---------- Seurat_PD_Gene_Figures
#----- PDgenes
# genes are not included if they aren't in the top variable genes
# (that means they aren't foudn in slot data, and errors occur.)
PDgenes_ASAP <- c("SNCA", "LRRK2", "VPS35", "GBA")
PDneurogenes <- c("ATP13A2", "ATXN2", "CHCHD2", "DNAJC6", "EIF4G1", 
                  "GBA", "GIGYF2", "LRRK2", "MAPT", "PRKN", "PARK7", "PINK1", 
                  "PLA2G6", "PRKAR1B", "RAB39B", "SNCA", "SYNJ1", "VPS35"
                  #"FBX07", "AGTR1"
                  )
extra_genes <- c('DDRGK1', 'DLG2', 'ELOVL7', 'FGD4', 'FYN', 'GPNMB', 'HIP1R', 'HLA-DRB5', 'IP6K2', 'ITPKB', 'KANSL1', 'NSF', 'PAM', 'RIMS1', 'RPS12', 'STK39', 'TMEM163', 'ZNF608')
PDgenes <- unique(c(PDgenes_ASAP, PDneurogenes, extra_genes))

for (PDgene in PDgenes){
  #----- Plot Expression: Overall
  # UMAP
  plot <- (FeaturePlot(object = seurat_anno, 
                      reduction = "umap",
                      features = PDgene,
                      order = TRUE) + 
            scale_colour_gradientn(colours = viridis(n = 11, 
                                                     direction = -1))) %>%
           LabelClusters(plot = ., id = "ident", repel = TRUE, box.padding = 1)
  print(plot)
  png(file=paste0("seurat_results/figures/Seurat_Annotation_PDgene_UMAP_", 
                  PDgene, 
                  ".png"), 
      width=600, 
      height=600)
  print(plot)
  dev.off()
  
  # Violin Plot
  plot <- VlnPlot(object = seurat_anno, 
                  features = PDgene,
                  pt.size = 0) + 
            theme(legend.position = "none")
  print(plot)
  png(file=paste0("seurat_results/figures/Seurat_Annotation_PDgene_Violin_", 
                  PDgene, 
                  ".png"), 
      width=600, 
      height=600)
  print(plot)
  dev.off()
  
  #----- Plot Expression: Separate by PD vs. HC
  # UMAP
  plot_seperate_features(seurat_anno, PDgene, ident = "GROUP", pt.size = .1, numcol = 2)
  ggsave(filename = paste0("seurat_results/figures/Seurat_Annotation_PDgene_UMAP_group_", 
                           PDgene, ".png"), 
         plot = plot_seperate_features(seurat_anno, PDgene, ident = "GROUP", pt.size = .05, numcol = 2),
         width = 600*2,
         height = 400*2,
         dpi = 150,
         units = "px"
         )
  
  # Violin Plot
  plot <- VlnPlot(object = seurat_anno, 
                  features = PDgene,
                  pt.size = 0,
                  split.by = "GROUP",
                  split.plot = TRUE)
  print(plot)
  png(file=paste0("seurat_results/figures/Seurat_Annotation_PDgene_Violin_group_", 
                  PDgene, 
                  ".png"), 
      width=600, 
      height=600)
  print(plot)
  dev.off()
}

## Seurat Cell Proportions

Celltype Proportion  
Count of Celltype per sample / Count of all Celltypes per sample  

In [ ]:
#---------- Seurat_Cell_Proportions
# beta regression models are designed for data between 0 and 1
# This give me percent differences
# emmeans(betafit,pairwise~Group)
# emmeans(betafit,pairwise~Group,type="response")
# To test that group is important, second way to be pvaule
# betafit1 <- betareg(Oligodendrocytes ~ Group, data = cellproportions)
# betafit2 <- betareg(Oligodendrocytes ~ 1, data = cellproportions)
# lrtest(betafit1,betafit2)
#----- Load data if needed
if (!exists("seurat_anno")) {
  seurat_anno <- readRDS(file = "seurat_results/seurat_anno.rds")
}

#----- Cell Proportions
cell_proportions <- seurat_anno@meta.data %>% 
  dplyr::count(cluster_annotation, GROUP, SAMPLE, AGE, SEX, PMI) %>% 
  dplyr::group_by(SAMPLE) %>% 
  dplyr::rename(Count = n) %>%
  dplyr::mutate(Proportion = Count / sum(Count)) %>% 
  dplyr::select(-Count) %>%
  tidyr::spread(cluster_annotation, Proportion) %>% 
  replace(is.na(.), 0)
#----- Save Results
write_delim(cell_proportions, 
            paste0("seurat_results/", NameProject, "_Cell_Proportions.txt"), 
            delim = "\t")

#---------- Cell Proportions differences
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Cell Proportions Differences - Group
cell_proportions_differences <- lapply(cell_types, function(cell_type) {
  fit_data <- cell_proportions %>% 
    dplyr::select(GROUP, SAMPLE, AGE, cell_type)
  fit_data1 <- fit_data %>% 
    dplyr::select(GROUP, SAMPLE, AGE, cell_type) %>%
    dplyr::rename(celltype = cell_type) %>% 
    dplyr::mutate(lemon_squeezer = (celltype * (10 - 1) + 0.5) / 10)
  fit_lm <- lm(formula = as.formula(paste(cell_type, "~ GROUP + AGE")), 
              data = fit_data)
  fit_beta <- betareg(formula = "lemon_squeezer ~ GROUP + AGE", 
                     data = fit_data1) 
  fit_results <- data.frame(Celltype = cell_type,
                   lm_pvalue = summary(fit_lm)$coefficients[2,4],
                   lm_diff = summary(fit_lm)$coefficients[2,1],
                   betareg_pvalue = summary(fit_beta)$coefficients$mean[2,4],
                   betareg_change = exp(summary(fit_beta)$coefficients$mean[2,1]))
  return(fit_results)
}) %>% bind_rows()
#----- Save Results
write_delim(cell_proportions_differences, 
            paste0("seurat_results/", NameProject, "_Cell_Proportions_Differences_Group.txt"), 
            delim = "\t")
#----- Cell Proportions Differences - Group (with PMI & SEX as covariates)
cell_proportions_differences_sex_pmi <- lapply(cell_types, function(cell_type) {
  fit_data <- cell_proportions %>% 
    dplyr::select(GROUP, SAMPLE, AGE, SEX, PMI, cell_type)
  fit_data1 <- fit_data %>% 
    dplyr::select(GROUP, SAMPLE, AGE, SEX, PMI, cell_type) %>%
    dplyr::rename(celltype = cell_type) %>% 
    dplyr::mutate(lemon_squeezer = (celltype * (10 - 1) + 0.5) / 10)
  fit_lm <- lm(formula = as.formula(paste(cell_type, "~ GROUP + AGE + SEX + PMI")), 
              data = fit_data)
  fit_beta <- betareg(formula = "lemon_squeezer ~ GROUP + AGE + SEX + PMI", 
                     data = fit_data1) 
  fit_results <- data.frame(Celltype = cell_type,
                   # (using more specific column names instead of indexes in case this changes)
                   lm_pvalue = summary(fit_lm)$coefficients["GROUPPD","Pr(>|t|)"],
                   lm_diff = summary(fit_lm)$coefficients["GROUPPD","Estimate"],
                   betareg_pvalue = summary(fit_beta)$coefficients$mean["GROUPPD","Pr(>|z|)"],
                   betareg_change = exp(summary(fit_beta)$coefficients$mean["GROUPPD","Estimate"]))
  return(fit_results)
}) %>% bind_rows()
#----- Save Results
write_delim(cell_proportions_differences_sex_pmi, 
            paste0("seurat_results/", NameProject,
                   "_Cell_Proportions_Differences_Group_PMI_Sex.txt"), 
            delim = "\t")

#---------- Cell Cycle Phase Proportions
cell_phase_proportions <- seurat_anno@meta.data %>% 
  dplyr::group_by(SAMPLE, AGE, SEX, PMI) %>% 
  dplyr::count(cluster_annotation, GROUP, SAMPLE, Phase) %>% 
  dplyr::rename(Count = n) %>%
  tidyr::spread(Phase, Count) %>%
  replace(is.na(.), 0) %>%
  dplyr::mutate(G1_Proportion = G1/(G1+G2M+S),
                G2M_Proportion = G2M/(G1+G2M+S),
                S_Proportion = S/(G1+G2M+S)) %>%
  dplyr::select(-G1,-G2M,-S) %>%  
  melt(id.vars=c("cluster_annotation", "GROUP", "SAMPLE", "AGE", "SEX", "PMI")) %>% 
  dplyr::rename(Phase = variable, 
                Proportion = value) %>% 
  tidyr::spread(cluster_annotation, Proportion)  %>% 
  replace(is.na(.), 0)
#----- Save Results
write_delim(cell_phase_proportions, 
            paste0("seurat_results/", NameProject, "_Cell_Proportions_Phase_Group.txt"), 
            delim = "\t")

#---------- Cell Cycle Phase Proportions differences
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Phase
phases <- c("G1_Proportion", "G2M_Proportion", "S_Proportion")
#----- Cell Cycle Phase Proportions Differences
cell_phase_proportions_differences <- lapply(cell_types, function(cell_type) {
  lapply(phases, function(phase){
    fit_data <- cell_phase_proportions %>% 
      dplyr::select(GROUP, SAMPLE, AGE, Phase, cell_type) %>% 
      dplyr::filter(Phase == phase)
    fit_data1 <- fit_data %>% 
      dplyr::rename(celltype = cell_type) %>% 
      dplyr::mutate(lemon_squeezer = (celltype * (10 - 1) + 0.5) / 10)
    fit_lm <- lm(formula = as.formula(paste(cell_type, "~ GROUP + AGE")), 
                 data = fit_data)
    fit_beta <- betareg(formula = "lemon_squeezer ~ GROUP + AGE", 
                        data = fit_data1) 
    fit_results <- data.frame(Celltype = cell_type,
                     Phase = phase,
                     lm_pvalue = summary(fit_lm)$coefficients[2,4],
                     lm_diff = summary(fit_lm)$coefficients[2,1],
                     betareg_pvalue = summary(fit_beta)$coefficients$mean[2,4],
                     betareg_change = exp(summary(fit_beta)$coefficients$mean[2,1]))
    return(fit_results)
  }) 
}) %>% bind_rows()
#----- Save Results
write_delim(cell_phase_proportions_differences, 
  paste0("seurat_results/", NameProject, "_Cell_Proportions_Phase_Differences_Group.txt"), 
  delim = "\t")
#----- Cell Cycle Phase Proportions Differences (with PMI & SEX as covariates)
cell_phase_proportions_differences_sex_pmi <- lapply(cell_types, function(cell_type) {
  lapply(phases, function(phase){
    fit_data <- cell_phase_proportions %>% 
      dplyr::select(GROUP, SAMPLE, AGE, SEX, PMI, Phase, cell_type) %>% 
      dplyr::filter(Phase == phase)
    fit_data1 <- fit_data %>% 
      dplyr::rename(celltype = cell_type) %>% 
      dplyr::mutate(lemon_squeezer = (celltype * (10 - 1) + 0.5) / 10)
    fit_lm <- lm(formula = as.formula(paste(cell_type, "~ GROUP + AGE + SEX + PMI")), 
                 data = fit_data)
    fit_beta <- betareg(formula = "lemon_squeezer ~ GROUP + AGE + SEX + PMI", 
                        data = fit_data1) 
    fit_results <- data.frame(Celltype = cell_type,
                     Phase = phase,
                     # (using more specific column names instead of indexes in case this changes)
                     lm_pvalue = summary(fit_lm)$coefficients["GROUPPD","Pr(>|t|)"],
                     lm_diff = summary(fit_lm)$coefficients["GROUPPD","Estimate"],
                     betareg_pvalue = summary(fit_beta)$coefficients$mean["GROUPPD","Pr(>|z|)"],
                     betareg_change = exp(summary(fit_beta)$coefficients$mean["GROUPPD","Estimate"]))
    return(fit_results)
  }) 
}) %>% bind_rows()
#----- Save Results
write_delim(cell_phase_proportions_differences_sex_pmi, 
  paste0("seurat_results/", NameProject, 
         "_Cell_Proportions_Phase_Differences_Group_PMI_Sex.txt"), 
  delim = "\t")


In [ ]:
#---------- Cell Proportions
cell_proportions <- 
  read_delim(paste0("seurat_results/", NameProject, "_Cell_Proportions.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
#----- Cell Proportions Data
cell_proportions %>%
  kable(caption = "Proportion of Cells per Cluster", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))
#----- Cell Proportions Figure by Sample
# to be plotted later -- 
# plot <- ggSample +
#   geom_boxplot(data = cell_proportions %>%
#                  pivot_longer(cols = Oligodendrocyte_Cells:Endothelial_Cells,
#                               names_to = "Celltype",
#                               values_to = "Proportion") %>%
#                  mutate(GROUP = as.factor(GROUP)
#                         # outlier = case_when(.$Proportion == 0)
#                         )
#                ,
#                aes(x = Celltype, y = Proportion, fill = GROUP)) +
#   ggtitle("Proportion of Cells per Cluster") +
#   theme(legend.position = "none") #+
#   # facet_wrap(~GROUP)
#   #geom_text_repel(aes(label = ))
#   # facet_wrap(~cluster_annotation, scale="free", ncol =4)
# plot

#----- FIGURES & TABLES BY GROUP
#----- Cell Proportions Figure
plot <- ggGroup + 
  geom_boxplot(data = cell_proportions %>% 
                        melt(id.vars=c("GROUP", "SAMPLE", "AGE", "SEX", "PMI")) %>% 
                        dplyr::rename(cluster_annotation = variable,
                                      Proportion = value),
               aes(x = GROUP, y = Proportion, fill = GROUP)) + 
  ggtitle("Proportion of Cells per Cluster") + 
  theme(legend.position = "none") + 
  facet_wrap(~cluster_annotation, scale="free", ncol =4)
plot
png(file="seurat_results/figures/Seurat_Cell_Proportions_group.png", 
    width=800, 
    height=600)
plot
dev.off()
ggsave(file="seurat_results/figures/Seurat_Cell_Proportions_group_high_res.png")
#----- Cell Proportions Differences
cell_proportions_differences <- read_delim(
  paste0("seurat_results/", NameProject, "_Cell_Proportions_Differences_Group.txt"), 
    delim = "\t", escape_double = FALSE, 
    trim_ws = TRUE)
#----- Cell Proportions Differences Data
cell_proportions_differences %>% 
  kable(caption = "Proportion of Cells per Cluster Differeneces", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))

#---------- Cell Cycle Phase Proportions
cell_phase_proportions <- 
  read_delim(paste0("seurat_results/", NameProject, "_Cell_Proportions_Phase_Group.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
#----- Cell Cycle Phase Proportions Data
cell_phase_proportions %>%
  kable(caption = "Proportion of Cells per Cluster", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))
#----- Cell Cycle Phase Proportions Figure
plot <- ggGroup + 
  geom_boxplot(data = cell_phase_proportions %>% 
                        melt(id.vars=c("GROUP", "SAMPLE", "AGE", "Phase", "SEX", "PMI")) %>% 
                        dplyr::rename(cluster_annotation = variable,
                                      Proportion = value),
               aes(x = Phase, y = Proportion, fill = GROUP)) + 
  ggtitle("Proportion of Cells per Cluster") + 
  # theme(legend.position = "none") + 
  facet_wrap(~cluster_annotation, scale="free", ncol =4)
plot
png(file="seurat_results/figures/Seurat_Cell_Proportions_Phase_group.png", 
    width=800, 
    height=600)
plot
dev.off()
#----- Cell Proportions Differences
cell_phase_proportions_differences <- read_delim(
  paste0("seurat_results/", NameProject, "_Cell_Proportions_Phase_Differences_Group.txt"), 
    delim = "\t", escape_double = FALSE, 
    trim_ws = TRUE)
#----- Cell Proportions Differences Data
cell_phase_proportions_differences %>% 
  kable(caption = "Proportion of Cells per Cluster Differeneces", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))


#---------- Data
#----- Remove Data
rm(cell_proportions_differences, 
   cell_phase_proportions, cell_phase_proportions_differences)

In [ ]:
#---------- Seurat_Cell_Proportions_Speckle
# install if package is not already installed.
if (!("speckle" %in% rownames(installed.packages()))) {
  remotes::install_github("Oshlack/speckle", 
                          build_vignettes = TRUE, 
                          dependencies = "Suggest")
}
library(BiocStyle)
library(CellBench)
library(limma)
library(scater)
library(speckle)
#----- Our Data
props <- getTransformedProps(clusters = seurat_anno$cluster_annotation, 
                             sample = seurat_anno$SAMPLE, 
                             transform = "logit")
barplot(props$Proportions, legend=TRUE, ylab="Proportions")
plot <- props$Proportions %>% 
  melt(id.vars=c("clusters")) %>% 
  dplyr::rename(Proportion = value) %>% 
  dplyr::mutate(clusters = as.character(clusters)) %>%
  dplyr::mutate(sample = as.character(sample)) %>%
  ggplot() + 
  geom_bar(aes(x = sample, y = Proportion, fill = clusters),
           position="fill", stat="identity") + 
  ggtitle("Proportion of Cells per Cluster") + 
  # scale_fill_brewer(palette = "Spectral") + 
  scale_fill_manual(values = hcl.colors(palette = "Spectral", n = 16)) +
  theme_classic() + 
  theme(legend.position = "right") +
  coord_flip()
plot
png(file="seurat_results/figures/Seurat_Cell_Proportions_group_Propeller.png", 
    width=800, 
    height=400)
plot
dev.off()
test_differences_propellor <- function(test_variable) {
  design <- model.matrix(as.formula(paste("~ 0 + ", test_variable, " + AGE")), 
                         data = cell_proportions)
  contrasts <- c(-1,1,0)
  
  # Determine & Conduct Test
  if ((ncol(design) - 1) == 2) {
    # two groups
    # mycontr <- makeContrasts(GROUPPD-GROUPHC, levels=design)
    test_type <- "t-test"
    props_test <- propeller.ttest(prop.list = props, 
                    design = design, 
                    contrasts = contrasts, 
                    robust = TRUE, 
                    trend = FALSE, 
                    sort = FALSE)
  } else if ((ncol(design) - 1) > 2) {
    # More than two groups
    # mycontr <- makeContrasts(GROUPPD-GROUPHC, levels=design)
    test_type <- "ANOVA"
    props_test <- propeller.anova(prop.list = props,
                                  design = design,
                                  coef = c(1,2,3),
                                  robust = TRUE,
                                  trend = FALSE,
                                  sort = FALSE)
  } else {
    stop("You have less than 2 comparison groups -- propeller won't work.")
  }
  # Return a named list with the stats table & the test type.
  list(props_test = props_test,
       test_type = test_type) %>%
    return()
  # return(props_test)
}
#----- Run Propeller
props_test <- test_differences_propellor("GROUP")
#----- Cell Proportions Table
kable(props_test$props_test, caption = paste("Proportion of Cells per Cluster (Propeller", 
                                             props_test$test_type, "test)"), 
      format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))
#----- Save Results
write_delim(props_test$props_test, 
  paste0("seurat_results/", NameProject, "_Cell_Proportions_Differences_Group_Propeller.txt"), 
  delim = "\t")
#---------- Data
#----- Remove Data
rm(cell_proportions)

In [ ]:
#----- Tutorial
library(speckle)
library(SingleCellExperiment)
library(CellBench)
library(limma)
library(ggplot2)
library(scater)
library(patchwork)
library(edgeR)
library(AnnotationDbi)
library(org.Hs.eg.db)
# browseVignettes("speckle")
# Single Cell data
sc_data <- load_sc_data()
# Common Genes
commongenes1 <- rownames(sc_data$sc_dropseq)[rownames(sc_data$sc_dropseq) %in% 
                                               rownames(sc_data$sc_celseq)]
commongenes2 <-  commongenes1[commongenes1 %in% rownames(sc_data$sc_10x)]
sce_10x <- sc_data$sc_10x[commongenes2,]
sce_celseq <- sc_data$sc_celseq[commongenes2,] 
sce_dropseq <- sc_data$sc_dropseq[commongenes2,] 
# Bootstrap additional samples
set.seed(10)
boot.10x <- sample(seq_len(ncol(sce_10x)), replace=TRUE)
boot.celseq <- sample(seq_len(ncol(sce_celseq)), replace=TRUE)
boot.dropseq <- sample(seq_len(ncol(sce_dropseq)), replace=TRUE)
sce_10x_rep2 <- sce_10x[,boot.10x]
sce_celseq_rep2 <- sce_celseq[,boot.celseq]
sce_dropseq_rep2 <- sce_dropseq[,boot.dropseq]
# Combine all SingleCellExperiment objects
allcounts <- cbind(counts(sce_10x),counts(sce_10x_rep2), 
                   counts(sce_celseq), counts(sce_celseq_rep2),
                   counts(sce_dropseq), counts(sce_dropseq_rep2))
sce_all <- SingleCellExperiment(assays = list(counts = allcounts))
sce_all$sample <- rep(c("S1","S2","S3","S4","S5","S6"), 
                      c(ncol(sce_10x), ncol(sce_10x_rep2), 
                        ncol(sce_celseq), ncol(sce_celseq_rep2), 
                        ncol(sce_dropseq), ncol(sce_dropseq_rep2)))
sce_all$group <- rep(c("10x","celseq","dropseq"),
                     c(2*ncol(sce_10x),2*ncol(sce_celseq),2*ncol(sce_dropseq)))
sce_all$cluster <- c(sce_10x$cell_line, sce_10x_rep2$cell_line, 
                     sce_celseq$cell_line, sce_celseq_rep2$cell_line, 
                     sce_dropseq$cell_line, sce_dropseq_rep2$cell_line)
rm(sc_data, commongenes1, commongenes2, sce_10x, sce_celseq, sce_dropseq, 
   boot.10x, boot.celseq, boot.dropseq, sce_10x_rep2, sce_celseq_rep2, sce_dropseq_rep2,
   allcounts)
sce_all <- scater::logNormCounts(sce_all)
sce_all <- scater::runPCA(sce_all)
sce_all <- scater::runUMAP(sce_all)
# Test for differences in cell line proportions in the three technologies
propeller(clusters = sce_all$cluster, 
          sample = sce_all$sample, 
          group = sce_all$group,
          robust = TRUE,
          transform = "logit")
propeller(clusters = sce_all$cluster, 
          sample = sce_all$sample, 
          group = sce_all$group,
          robust = TRUE,
          transform = "asin")
# Fitting linear models using the transformed proportions directly
plotCellTypeProps(sce_all)
props <- getTransformedProps(sce_all$cluster, sce_all$sample, transform="logit")
barplot(props$Proportions, col = c("orange","purple","dark green"),legend=TRUE, 
        ylab="Proportions")
group <- rep(c("10x","celseq","dropseq"),each=2)
pair <- rep(c(1,2),3)
design <- model.matrix(~ 0 + group + pair)
propeller.anova(prop.list=props, 
                design=design, 
                coef = c(1,2,3), 
                robust=TRUE, 
                trend=FALSE, 
                sort=TRUE)
mycontr <- makeContrasts(group10x-groupdropseq, levels=design)
propeller.ttest(prop.list = props, 
                design = design, 
                contrasts = mycontr, 
                robust=TRUE, 
                trend=FALSE, 
                sort=TRUE)

## Seurat Conserved Markers
  
Identification of all markers for each cluster  
This type of analysis is typically recommended for when evaluating a single  
sample group/condition. With the FindAllMarkers() function we are comparing each  
cluster against all other clusters to identify potential marker genes. The cells  
in each cluster are treated as replicates, and essentially a differential  
expression analysis is performed with some statistical test. Also, by default  
this function will return to you genes that exhibit both positive and negative  
expression changes. Typically, we add an argument only.pos to opt for keeping 
only the positive changes.  
  
Identification of conserved markers in all conditions  
Since we have samples representing different conditions in our dataset, our best  
option is to find conserved markers. This function internally separates out  
cells by sample group/condition, and then performs differential gene expression  
testing for a single specified cluster against all other clusters (or a second  
cluster, if specified). Gene-level p-values are computed for each condition and  
then combined across groups using meta-analysis methods from the MetaDE R  
package. Before we start our marker identification we will explicitly set our  
default assay, we want to use the original counts and not the integrated data.  
DefaultAssay(seurat_anno) <- "RNA"  

In [ ]:
#---------- Seurat_Conserved_Markers
# RNA stores the original and normalized counts for finding markers
# DefaultAssay(seurat_anno) <- "RNA"
# Ensembl & Symbol Gene IDs 
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Conserved Markers across sample groups
get_conserved <- function(seurat_obj, cell_type){
  FindConservedMarkers(object = seurat_obj,
                       ident.1 = cell_type, 
                       grouping.var = "GROUP", 
                       only.pos = TRUE, 
                       min.diff.pct = 0.1, 
                       min.pct = 0.1, 
                       logfc.threshold = 0.25) %>% 
    rownames_to_column(var = "Gene_Symbol") %>% 
    left_join(Gene_Emsembl_Symbol, by="Gene_Symbol") %>% 
    dplyr::mutate(Celltype = cell_type)
}
Conserved_Markers <- mclapply(cell_types, function(i) {
  print(i)
  get_conserved(seurat_obj = seurat_anno,
                cell_type = i) 
}) %>% bind_rows()
#---- Save Results
write_delim(Conserved_Markers, 
            paste0("seurat_results/", NameProject, "_Conserved_Markers.txt"), 
            delim = "\t")

#---------- Conserved Markers Pathways
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Conserved Markers Pathways
Conserved_Markers_Pathways <- lapply(cell_types, function(cell_type) {
  genes <- Conserved_Markers %>% 
             dplyr::filter(Celltype == cell_type) %>% 
             dplyr::pull(Gene_Symbol)
  genes_go <- enrichGO(gene = genes,
                       OrgDb = "org.Hs.eg.db",
                       ont ="BP",
                       keyType = "SYMBOL",
                       pAdjustMethod = "BH")
  return(genes_go@result %>% mutate(Celltype = cell_type))
}) %>% bind_rows()
#----- Save Results
write_delim(Conserved_Markers_Pathways, 
  paste0("seurat_results/", NameProject, "_Conserved_Markers_Pathways_GOBP.txt"), 
  delim = "\t")

In [ ]:
#---------- Seurat_Conserved_Markers_Figures
Conserved_Markers <- 
  read_delim(paste0("seurat_results/", NameProject, "_Conserved_Markers.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
#----- Conserved Markers Heatmap
# Idents(seurat_anno) <- seurat_anno$cluster_annotation
plot <- DoHeatmap(object = seurat_anno, 
          features = Conserved_Markers %>% 
                       dplyr::group_by(Celltype) %>% 
                       dplyr::mutate(avg_log2FC = (HC_avg_log2FC + 
                                                   PD_avg_log2FC)/2) %>% 
                       top_n(10, avg_log2FC) %>% 
                       dplyr::pull(Gene_Symbol),
          label = TRUE)
plot
png(file="seurat_results/figures/Seurat_Conserved_Markers_heatmap.png", 
    width=1000, 
    height=1000)
plot
dev.off()


#---------- Conserved Markers Pathways Figure
Conserved_Markers_Pathways <- 
  read_delim(paste0("seurat_results/", NameProject, "_Conserved_Markers_Pathways_GOBP.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Conserved Markers Pathways
for (cell_type in cell_types){
  genes <- Conserved_Markers %>% 
             dplyr::filter(Celltype == cell_type) %>% 
             dplyr::pull(Gene_Symbol)
  genes_go <- enrichGO(gene = genes,
                       OrgDb = "org.Hs.eg.db",
                       ont ="BP",
                       keyType = "SYMBOL",
                       pAdjustMethod = "BH")
  plot <- enrichplot::dotplot(genes_go, showCategory=20) + 
            ggtitle(paste0(cell_type, " - GO:Biological Process")) + 
            theme(plot.title = element_text(hjust = 0.5, 
                                            face = "bold"))
  print(plot)
  png(file=
    paste0("seurat_results/figures/Seurat_Conserved_Markers_Pathways_GOBP_", 
           cell_type, ".png"), 
      width=1000, 
      height=600)
  print(plot)
  dev.off()
  plot <- genes_go %>% pairwise_termsim() %>% 
            enrichplot::emapplot(showCategory=40) + 
              ggtitle(paste0(cell_type, " - GO:Biological Process")) + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold"))
  print(plot)
  png(file=
    paste0("seurat_results/figures/Seurat_Conserved_Markers_Pathways_GOBPenrich_", 
           cell_type, ".png"), 
      width=1000, 
      height=1000)
  print(plot)
  dev.off()
}  
  

## Seurat Differential Expression

Find DEG between PD and HC per cell_type  
Using 0.25 logFC  

In [ ]:
#---------- Seurat_Differential_Expression
#----- RNA Assay stores the original and normalized counts
# Original counts : seurat_anno@assays[["RNA"]]@counts
# Normalized counts : seurat_anno@assays[["RNA"]]@data
DefaultAssay(seurat_anno) <- "RNA"
#----- Group Celltype ID
# make this the Idents()
seurat_anno$group_cluster <- paste(seurat_anno$GROUP, 
                                   seurat_anno$cluster_annotation, 
                                   sep = "_")
Idents(seurat_anno) <- seurat_anno$group_cluster

#---------- Differential Expressed Genes
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Cell_Type Differential Expressed Genes
get_deg <- function(seurat_obj, cell_type, ident1, ident2){
  FindMarkers(seurat_obj, 
              ident.1 = paste(ident1, cell_type, sep = "_"), 
              ident.2 = paste(ident2, cell_type, sep = "_"), 
              test.use = "MAST", 
              latent.vars = "AGE", 
              logfc.threshold = 0, 
              min.pct = 0,
              min.diff.pct = -Inf, 
              min.cells.feature = 10, 
              min.cells.group = 10, 
              verbose = FALSE) %>% 
    rownames_to_column(var = "Gene_Symbol") %>% 
    left_join(Gene_Emsembl_Symbol, by="Gene_Symbol") %>% 
    mutate(Comparison = paste(ident1, "vs", ident2, cell_type, sep = "_"))
}
Differential_Expression <- lapply(cell_types, function(cell_type) {
  tryCatch(get_deg(seurat_obj = seurat_anno,
                   cell_type = cell_type,
                   ident1 = "PD",
                   ident2 = "HC"),
           error = function(e) 
             # empty dataframe with same column headers & types...
             data.frame(Gene_Symbol = character(),
                        p_val = double(),
                        avg_log2FC = double(),
                        pct.1 = double(),
                        pct.2 = double(),
                        p_val_adj = double(),
                        Gene_Emsembl = character(),
                        Comparison = character(),
                        stringsAsFactors = FALSE))
}) %>% bind_rows()
#----- save the results
write_delim(Differential_Expression, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST.txt"), 
  delim = "\t")
saveRDS(Differential_Expression, file = "seurat_results/differential_expression.rds")
#----- Number of DEGs
Differential_Expression_count <- Differential_Expression %>% 
  mutate(Significant = case_when(
           p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
           p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
           TRUE ~ 'Not_Significant')) %>% 
  dplyr::count(Comparison, Significant) %>% 
  dplyr::rename(Count = n) %>% 
  tidyr::spread(Significant, Count)
#----- save the results
write_delim(Differential_Expression_count, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_count.txt"), 
  delim = "\t")


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
Differential_Expression_GOBP <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  return(genes_go@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_GOBP, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_Pathways_GSEA_GOBP.txt"), 
  delim = "\t")
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
Differential_Expression_REACTOME <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  return(genes_gsea@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_REACTOME, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_Pathways_GSEA_REACTOME.txt"), 
  delim = "\t")

In [ ]:
#---------- Seurat_Differential_Expression_Figures
Differential_Expression <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)

#----- Differential_Expression_count
Differential_Expression_count <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_count.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
Differential_Expression_count %>% 
  kable(caption="Number of Differentially Expressed Genes", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))


#---------- Differential Expressed Genes Volcano
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Differential Expressed Genes Volcano
for (comparison in comparisons){
  plot <- Differential_Expression %>% 
            dplyr::filter(Comparison == comparison) %>%
            dplyr::select(Comparison, 
                          Gene_Symbol, 
                          avg_log2FC, 
                          p_val,
                          p_val_adj) %>% 
            mutate(Significant = case_when(
                     p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
                     p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
                     TRUE ~ 'Not_Significant')) %>% 
            mutate(Label = case_when(
                     Significant == "Upregulated" ~ Gene_Symbol,
                     Significant == "Downregulated" ~ Gene_Symbol,
                     TRUE ~ NA_character_)) %>% 
            ggplot(mapping = aes(x=avg_log2FC, 
                   y = -log10(p_val), 
                   color = Significant,
                   label = Label)) + 
              geom_point(alpha = 0.5, stroke=0, size=2) + 
              geom_text_repel() +
              ggtitle(comparison) + 
              xlab("log2 Fold Change") +
              ylab("-log10(P Value)") + 
              theme_classic() + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold")) + 
              scale_colour_manual(values = c('Blue','Gray','Red')) + 
              geom_vline(xintercept = c(-0.25,0.25), 
                         linetype = "dotted")
  print(plot)
  # png(file = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
  #   width = 600*2, 
  #   height = 400*2,
  #   pointsize = 6)
  # print(plot)
  # dev.off()
  ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
         plot = plot,
         width = 600*2,
         height = 400*2,
         dpi = 150,
         units = "px")
}


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  if (nrow(genes_go@result) == 0) {
     warning(paste0("No significant GOBP genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
     plot <- enrichplot::dotplot(object = genes_go, 
                                  showCategory = 10, 
                                  split = ".sign") + 
                facet_grid(.~.sign) + 
                ggtitle(paste0(comparison, " - GO:Biological Process")) + 
                theme(plot.title = element_text(hjust = 0.5, 
                                                face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaGOBP.png", sep = "_"), 
          width=1000, 
          height=600)
      print(plot)
      dev.off()
   } 
  
  #----- Split GOBP Pathway Map by Activated / Suppressed
  # (otherwise the activated & suppressed are shown together -- misleading)
  genes_go_emapplot <- genes_go
  genes_go_emapplot@result <- genes_go_emapplot@result %>%
    mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                         NES < 0 ~ "suppressed"))
  genes_go_emapplot_activated <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "activated")
  genes_go_emapplot_suppressed <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "suppressed")
  
  # plot activated
  if (nrow(genes_go_emapplot_activated@result) == 0) {
     warning(paste0("No significant activated GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_activated <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_activated <- pairwise_termsim(x = genes_go_emapplot_activated)
    plot_activated <- genes_go_emapplot_activated %>% 
              enrichplot::emapplot(showCategory=40) +
              theme(plot.title = element_text(hjust = 0.5,
                                              face = "bold"))
    print(plot_activated)
  }
  
  # plot suppressed
  if (nrow(genes_go_emapplot_suppressed@result) == 0) {
     warning(paste0("No significant suppressed GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_suppressed <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_suppressed <- pairwise_termsim(x = genes_go_emapplot_suppressed)
    plot_suppressed <- genes_go_emapplot_suppressed %>% 
              enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
    print(plot_suppressed)
  } 
  
  # combine & save plot
  sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ncol = 2)
  
  
  plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                     " - GO:Biological Process"),
                                                     size = 24,
                                                     hjust = 0.5,
                                                     fontface = "bold"),
                             sub_plot_labels,
                             plot_grid(plot_activated, plot_suppressed),
                             ncol = 1, rel_heights = c(0.1, .05, 1))

  plot
  
  png(file=
    paste("seurat_results/figures/Seurat_Differential_Expression", 
           comparison, "AGE_MAST_Pathways_gseaGOBPenrich.png", sep = "_"), 
      width=1400, 
      height=800)
  print(plot)
  dev.off()
  #----- pathExplore: High-Level GSEA Clusters (GO:BP)
  tryCatch({
    # fix here: https://github.com/tidyverse/ggplot2/issues/2799
    cf <- coord_fixed()
    cf$default <- TRUE
    plot <- enrichmentNetwork(genes_go@result, drawEllipses = TRUE, fontSize = 2.5) +
      ggtitle(paste0(comparison, " - GO:Biological Process")) +
      theme(plot.title = element_text(hjust = 0.5, face = "bold"),
            plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
      cf +
      coord_fixed(clip = "off")
    print(plot)
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                            comparison, "AGE_MAST_Pathways_gseaGOBPenrich_pathExplore.png", 
                            sep = "_"),
           plot = plot,
           width = 600*2,
           height = 400*2,
           dpi = 150,
           units = "px")
  },
    error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
}
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  if (nrow(genes_gsea@result) == 0) {
    warning(paste0("No significant REACTOME genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
    plot <- enrichplot::dotplot(object = genes_gsea,#[genes_gsea@result$ID %in% c("REACTOME_SENESCENCE_ASSOCIATED_SECRETORY_PHENOTYPE_SASP", "REACTOME_CELLULAR_RESPONSE_TO_HEAT_STRESS", "REACTOME_PROGRAMMED_CELL_DEATH", "REACTOME_NUCLEOTIDE_EXCISION_REPAIR", "REACTOME_LAMININ_INTERACTIONS", "REACTOME_EXTRACELLULAR_MATRIX_ORGANIZATION"), asis = TRUE], #REACTOME_SIGNALING_BY_INTERLEUKINS, REACTOME_RRNA_PROCESSING, REACTOME_SIGNALING_BY_NOTCH4
                                showCategory = 10, 
                                split = ".sign") + 
              facet_grid(.~.sign) + 
              ggtitle(paste0(comparison, " - GSEA Reactome")) + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold"))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME.png", sep = "_"), 
        width=1000, 
        height=600)
    print(plot)
    dev.off()
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME_quality.png", sep = "_"), 
       plot = plot,
       width = 1000*2,
       height = 600*2,
       dpi = 200,
       units = "px")
    
    #----- Split REACTOME Pathway Map by Activated / Suppressed
    # (otherwise the activated & suppressed are shown together -- misleading)
    genes_gsea_emapplot <- genes_gsea
    genes_gsea_emapplot@result <- genes_gsea_emapplot@result %>%
      mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                           NES < 0 ~ "suppressed"))
    genes_gsea_emapplot_activated <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "activated")
    genes_gsea_emapplot_suppressed <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "suppressed")
    
    # plot activated
    if (nrow(genes_gsea_emapplot_activated@result) == 0) {
       warning(paste0("No significant activated REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_activated <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_activated <- pairwise_termsim(x = genes_gsea_emapplot_activated)
      plot_activated <- genes_gsea_emapplot_activated %>% 
                enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
      print(plot_activated)
    }
    
    # plot suppressed
    if (nrow(genes_gsea_emapplot_suppressed@result) == 0) {
       warning(paste0("No significant suppressed REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_suppressed <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_suppressed <- pairwise_termsim(x = genes_gsea_emapplot_suppressed)
      plot_suppressed <- genes_gsea_emapplot_suppressed %>% 
                enrichplot::emapplot(showCategory=40) +
                  theme(plot.title = element_text(hjust = 0.5,
                                                  face = "bold"))
      print(plot_suppressed)
    } 
    
    # combine & save plot
    sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ncol = 2)
    
    
    plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                       " - GSEA Reactome"),
                                                       size = 24,
                                                       hjust = 0.5,
                                                       fontface = "bold"),
                               sub_plot_labels,
                               plot_grid(plot_activated, plot_suppressed),
                               ncol = 1, rel_heights = c(0.1, .05, 1))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich.png", sep = "_"), 
        width=1400, 
        height=800)
    print(plot)
    dev.off()
    #----- pathExplore: High-Level GSEA Clusters (GSEA Reactome)
    tryCatch({
      # fix here: https://github.com/tidyverse/ggplot2/issues/2799
      cf <- coord_fixed()
      cf$default <- TRUE
      plot <- pathExplore::enrichmentNetwork(genes_gsea@result, drawEllipses = TRUE, fontSize = 2.5) +
        ggtitle(paste0(comparison, " - GSEA Reactome")) +
        theme(plot.title = element_text(hjust = 0.5, face = "bold"),
              plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
        cf +
        coord_fixed(clip = "off")
      print(plot)
      ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                              comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich_pathExplore.png",
                              sep = "_"),
             plot = plot,
             width = 600*2,
             height = 400*2,
             dpi = 150,
             units = "px")
    },
      error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
   } 
}


## Seurat Differential Expression: Individual Sex

Sex = 1: Male  
Sex = 2: Female  

Run the DEG workflow for all our Males, then all our Females. 

(Code displayed, but not run for this knitted html. 
This was used to check something previously.)


In [ ]:
#---------- Seurat_Differential_Expression
do_all_differential_expression <- function(sex, seurat_anno) {
  #----- RNA Assay stores the original and normalized counts
  # Original counts : seurat_anno@assays[["RNA"]]@counts
  # Normalized counts : seurat_anno@assays[["RNA"]]@data
  DefaultAssay(seurat_anno) <- "RNA"
  #----- Subset by Sex
  # subset to only our Sex of interest.
  if (sex %in% c("M", "Male", "male", "m")) {
    sex_coded <- 1
    sex <- "M"
  } else if (sex %in% c("F", "Female", "female", "f")) {
    sex_coded <- 2
    sex <- "F"
  } else {
    stop(paste0("Invalid `sex` argument. You provided: `sex` = ", sex,
                "\n`sex` must be one of the following: ",
                '\nfor Males: M, Male, male, or m',
                '\nfor Females: F, Female, female, f'))
  }
  seurat_anno <- subset(seurat_anno, subset = SEX %in% sex_coded)
  #----- Group Celltype ID
  # make this the Idents()
  seurat_anno$group_cluster <- paste(seurat_anno$GROUP, 
                                     seurat_anno$cluster_annotation, 
                                     sep = "_")
  Idents(seurat_anno) <- seurat_anno$group_cluster
  
  #---------- Differential Expressed Genes
  #----- Cell Types
  cell_types <- seurat_anno@meta.data %>% 
    dplyr::arrange(cluster_annotation) %>% 
    dplyr::select(cluster_annotation) %>% 
    unique() %>% 
    pull(cluster_annotation)
  #----- Cell_Type Differential Expressed Genes
  get_deg <- function(seurat_obj, cell_type, ident1, ident2){
    FindMarkers(seurat_obj, 
                ident.1 = paste(ident1, cell_type, sep = "_"), 
                ident.2 = paste(ident2, cell_type, sep = "_"), 
                test.use = "MAST", 
                latent.vars = "AGE", 
                logfc.threshold = 0, 
                min.pct = 0,
                min.diff.pct = -Inf, 
                min.cells.feature = 10, 
                min.cells.group = 10, 
                verbose = FALSE) %>% 
      rownames_to_column(var = "Gene_Symbol") %>% 
      left_join(Gene_Emsembl_Symbol, by="Gene_Symbol") %>% 
      mutate(Comparison = paste(ident1, "vs", ident2, cell_type, sep = "_"))
  }
  Differential_Expression <- lapply(cell_types, function(cell_type) {
  tryCatch(get_deg(seurat_obj = seurat_anno,
                   cell_type = cell_type,
                   ident1 = "PD",
                   ident2 = "HC"),
           error = function(e) 
             # empty dataframe with same column headers & types...
             data.frame(Gene_Symbol = character(),
                        p_val = double(),
                        avg_log2FC = double(),
                        pct.1 = double(),
                        pct.2 = double(),
                        p_val_adj = double(),
                        Gene_Emsembl = character(),
                        Comparison = character(),
                        stringsAsFactors = FALSE))
  }) %>% bind_rows()
  #----- save the results
  write_delim(Differential_Expression, 
    paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_", sex, ".txt"), 
    delim = "\t")
  
  #----- Number of DEGs
  Differential_Expression_count <- Differential_Expression %>% 
    mutate(Significant = case_when(
             p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
             p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
             TRUE ~ 'Not_Significant')) %>% 
    dplyr::count(Comparison, Significant) %>% 
    dplyr::rename(Count = n) %>% 
    tidyr::spread(Significant, Count)
  #----- save the results
  write_delim(Differential_Expression_count, 
    paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_count_", sex, ".txt"), 
    delim = "\t")
  
  
  #---------- Differential Expressed Genes Pathways
  #----- Comparisons
  comparisons <- Differential_Expression$Comparison %>% 
                   unique()
  #----- Pathways GSEA GOBP
  Differential_Expression_GOBP <- lapply(comparisons, function(comparison) {
    genes <- Differential_Expression %>% 
      dplyr::filter(Comparison == comparison) %>%
      dplyr::select(Comparison, 
                    Gene_Symbol, 
                    avg_log2FC, 
                    p_val) %>%
      dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                      TRUE ~ p_val)) %>%
      dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
      dplyr::arrange(sign_log10Pvalue)
    genelist <- setNames(genes$sign_log10Pvalue, 
                         genes$Gene_Symbol) %>% 
      sort(decreasing = TRUE)
    genes_go <- gseGO(geneList = genelist, 
                      ont ="BP", 
                      OrgDb = get("org.Hs.eg.db"),
                      keyType = "SYMBOL", 
                      minGSSize = 10, 
                      maxGSSize = 500, 
                      pvalueCutoff = 0.05, 
                      pAdjustMethod = "BH",
                      verbose = TRUE)
    return(genes_go@result %>% mutate(Comparison = comparison))
  }) %>% bind_rows()
  #----- Save Results
  write_delim(Differential_Expression_GOBP, 
    paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_Pathways_GSEA_GOBP_", sex, ".txt"), 
    delim = "\t")
  #----- Reactome
  reactome <- msigdbr(species = "Homo sapiens", 
                      category = "C2", 
                      subcategory = "CP:REACTOME") %>% 
                dplyr::select(gs_name, gene_symbol)
  #----- Pathways GSEA REACTOME
  Differential_Expression_REACTOME <- lapply(comparisons, function(comparison) {
    genes <- Differential_Expression %>% 
      dplyr::filter(Comparison == comparison) %>%
      dplyr::select(Comparison, 
                    Gene_Symbol, 
                    avg_log2FC, 
                    p_val) %>%
      dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                      TRUE ~ p_val)) %>%
      dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
      dplyr::arrange(sign_log10Pvalue)
    genelist <- setNames(genes$sign_log10Pvalue, 
                         genes$Gene_Symbol) %>% 
      sort(decreasing = TRUE)
    genes_gsea <- GSEA(geneList = genelist,
                       minGSSize = 10,
                       maxGSSize = 500,
                       pvalueCutoff = 0.05,
                       pAdjustMethod = "BH",
                       TERM2GENE = reactome, 
                       by = "fgsea",
                       verbose = TRUE)
    return(genes_gsea@result %>% mutate(Comparison = comparison))
  }) %>% bind_rows()
  #----- Save Results
  write_delim(Differential_Expression_REACTOME, 
    paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_Pathways_GSEA_REACTOME_", sex, ".txt"), 
    delim = "\t")
}

#----- Load data if needed
if (!exists("seurat_anno")) {
  seurat_anno <- readRDS(file = "seurat_results/seurat_anno.rds")
}
#----- Do differential expression on each of the individual sexes
do_all_differential_expression(sex = "F", 
                               seurat_anno = seurat_anno)
do_all_differential_expression(sex = "M",
                               seurat_anno = seurat_anno)

In [ ]:
#---------- Seurat_Differential_Expression_Figures
do_all_differential_expression_figures <- function(sex) {
  #----- Process function arguments
  if (sex %in% c("M", "Male", "male", "m")) {
    sex_coded <- 1
    sex <- "M"
  } else if (sex %in% c("F", "Female", "female", "f")) {
    sex_coded <- 2
    sex <- "F"
  } else {
    stop(paste0("Invalid `sex` argument. You provided: `sex` = ", sex,
                "\n`sex` must be one of the following: ",
                '\nfor Males: M, Male, male, or m',
                '\nfor Females: F, Female, female, f'))
  }
  
  #----- Differential_Expression
  Differential_Expression <- 
    read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_", sex, ".txt"), 
    delim = "\t", escape_double = FALSE, 
    trim_ws = TRUE)
  
  #----- Differential_Expression_count
  Differential_Expression_count <- 
    read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_PD_vs_HC_AGE_MAST_count_", sex, ".txt"), 
    delim = "\t", escape_double = FALSE, 
    trim_ws = TRUE)
  Differential_Expression_count %>% 
    kable(caption="Number of Differentially Expressed Genes", 
          format.args = list(big.mark = ",")) %>% 
    kable_styling(c("striped", "bordered"))
  
  
  #---------- Differential Expressed Genes Volcano
  #----- Comparisons
  comparisons <- Differential_Expression$Comparison %>% 
                   unique()
  #----- Differential Expressed Genes Volcano
  for (comparison in comparisons){
    plot <- Differential_Expression %>% 
              dplyr::filter(Comparison == comparison) %>%
              dplyr::select(Comparison, 
                            Gene_Symbol, 
                            avg_log2FC, 
                            p_val,
                            p_val_adj) %>% 
              mutate(Significant = case_when(
                       p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
                       p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
                       TRUE ~ 'Not_Significant')) %>% 
              mutate(Label = case_when(
                       Significant == "Upregulated" ~ Gene_Symbol,
                       Significant == "Downregulated" ~ Gene_Symbol,
                       TRUE ~ NA_character_)) %>% 
              ggplot(mapping = aes(x=avg_log2FC, 
                     y = -log10(p_val), 
                     color = Significant,
                     label = Label)) + 
                geom_point(alpha = 0.5, stroke=0, size=2) + 
                geom_text_repel() +
                ggtitle(paste0(comparison, "\n", sex)) + 
                xlab("log2 Fold Change") +
                ylab("-log10(P Value)") + 
                theme_classic() + 
                theme(plot.title = element_text(hjust = 0.5, 
                                                face = "bold")) + 
                scale_colour_manual(values = c('Downregulated' = 'Blue',
                                               'Not_Significant' = 'Gray',
                                               'Upregulated' = 'Red')) + 
                geom_vline(xintercept = c(-0.25,0.25), 
                           linetype = "dotted")
    print(plot)
    # png(file = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
    #   width = 600*2, 
    #   height = 400*2,
    #   pointsize = 6)
    # print(plot)
    # dev.off()
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano", paste0(sex, ".png"), sep = "_"),
           plot = plot,
           width = 600*2,
           height = 400*2,
           dpi = 150,
           units = "px")
  }
  
  
  #---------- Differential Expressed Genes Pathways
  #----- Comparisons
  comparisons <- Differential_Expression$Comparison %>% 
                   unique()
  #----- Pathways GSEA GOBP
  for (comparison in comparisons){
    genes <- Differential_Expression %>% 
      dplyr::filter(Comparison == comparison) %>%
      dplyr::select(Comparison, 
                    Gene_Symbol, 
                    avg_log2FC, 
                    p_val) %>%
      dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                      TRUE ~ p_val)) %>%
      dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
      dplyr::arrange(sign_log10Pvalue)
    genelist <- setNames(genes$sign_log10Pvalue, 
                         genes$Gene_Symbol) %>% 
      sort(decreasing = TRUE)
    genes_go <- gseGO(geneList = genelist, 
                      ont ="BP", 
                      OrgDb = get("org.Hs.eg.db"),
                      keyType = "SYMBOL", 
                      minGSSize = 10, 
                      maxGSSize = 500, 
                      pvalueCutoff = 0.05, 
                      pAdjustMethod = "BH",
                      verbose = TRUE)
    if (nrow(genes_go@result) == 0) {
      # Error if this is empty & if we try to continue. Manual error handling...
        # Error in `$<-.data.frame`(`*tmp*`, ".sign", value = "activated") : 
        # replacement has 1 row, data has 0
      warning(paste0("No Significant Pathways...\n",
                     "Skipping plot GSEA GOBP plot for: ", comparison, " (", sex, ")"))
    } else {
      plot <- enrichplot::dotplot(object = genes_go, 
                                  showCategory = 10, 
                                  split = ".sign") + 
                facet_grid(.~.sign) + 
                ggtitle(paste0(comparison, " - GO:Biological Processs\n", sex)) + 
                theme(plot.title = element_text(hjust = 0.5, 
                                                face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaGOBP", paste0(sex, ".png"), sep = "_"), 
          width=1000, 
          height=600)
      print(plot)
      dev.off()
      plot <- pairwise_termsim(x = genes_go) %>% 
                enrichplot::emapplot(showCategory=40) + 
                  ggtitle(paste0(comparison, " - GO:Biological Process\n", sex)) + 
                  theme(plot.title = element_text(hjust = 0.5, 
                                                  face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaGOBPenrich", paste0(sex, ".png"), sep = "_"), 
          width=1400, 
          height=800)
      print(plot)
      dev.off()
      }
  }
  #----- Reactome
  reactome <- msigdbr(species = "Homo sapiens", 
                      category = "C2", 
                      subcategory = "CP:REACTOME") %>% 
                dplyr::select(gs_name, gene_symbol)
  #----- Pathways GSEA REACTOME
  for (comparison in comparisons){
    genes <- Differential_Expression %>% 
      dplyr::filter(Comparison == comparison) %>%
      dplyr::select(Comparison, 
                    Gene_Symbol, 
                    avg_log2FC, 
                    p_val) %>%
      dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                      TRUE ~ p_val)) %>%
      dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
      dplyr::arrange(sign_log10Pvalue)
    genelist <- setNames(genes$sign_log10Pvalue, 
                         genes$Gene_Symbol) %>% 
      sort(decreasing = TRUE)
    genes_gsea <- GSEA(geneList = genelist,
                       minGSSize = 10,
                       maxGSSize = 500,
                       pvalueCutoff = 0.05,
                       pAdjustMethod = "BH",
                       TERM2GENE = reactome, 
                       by = "fgsea",
                       verbose = TRUE)
    if (nrow(genes_gsea@result) == 0) {
      # Error if this is empty & if we try to continue. Manual error handling...
        # Error in `$<-.data.frame`(`*tmp*`, ".sign", value = "activated") : 
        # replacement has 1 row, data has 0
      warning(paste0("No Significant Pathways...\n",
                     "Skipping plot GSEA Reactome plot for: ", comparison, " (", sex, ")"))
    } else {
      plot <- enrichplot::dotplot(object = genes_gsea, 
                                  showCategory = 10, 
                                  split = ".sign") + 
                facet_grid(.~.sign) + 
                ggtitle(paste0(comparison, " - GSEA Reactome\n", sex)) + 
                theme(plot.title = element_text(hjust = 0.5, 
                                                face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaREACTOME", paste0(sex, ".png"), sep = "_"), 
          width=1000, 
          height=600)
      print(plot)
      dev.off()
      plot <- pairwise_termsim(x = genes_gsea) %>% 
                enrichplot::emapplot(showCategory=40) + 
                  ggtitle(paste0(comparison, " - GSEA Reactome\n", sex)) + 
                  theme(plot.title = element_text(hjust = 0.5, 
                                                  face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich", paste0(sex, ".png"), sep = "_"), 
          width=1400, 
          height=800)
      print(plot)
      dev.off()
    } 
  }
}
do_all_differential_expression_figures(sex = "F")
do_all_differential_expression_figures(sex = "M")

## Export Main Data (Vugene)

In [ ]:
#---------- Export_Data_Vugene
#----- Load Anno Object
if (!exists("seurat_anno")) {
  seurat_anno <- readRDS(file = "seurat_results/seurat_anno.rds")
}

#----- Get Data: Counts
# ?FetchData
# raw_counts_vugene <- FetchData(object = seurat_anno,
#                                # need vars... I think I want everything
#                                slot = "counts") # default is data, I want counts
# ?GetAssayData()
raw_counts_vugene <- GetAssayData(object = seurat_anno, 
                                  slot = "counts" # default is data, I want counts
                                  ) # I only have one assay (RNA) so I won't specify

#----- Get Data: Annotations
annotations_vugene <- dplyr::select(seurat_anno@meta.data, cluster_annotation)

#----- Save Data
saveRDS(raw_counts_vugene, 
        file=paste0("seurat_results/", NameProject, "_raw_counts_vugene.rds"))
# write.table(as.matrix(raw_counts_vugene[1:3,1:3]),
#             file = "seurat_results/PD_MFG_snRNAseq_with_redos_raw_counts_vugene.txt",
#             quote = FALSE)

write.table(annotations_vugene,
            sep = "\t",
            quote = FALSE,
            col.names = FALSE,
            file = paste0("seurat_results/", NameProject, "_annotations_vugene.tsv"))

## Seurat Differential Expression: p16+ / p16-

Find DEG between PD and HC per cell_type
Run the DEG workflow between p16+ and p16- cells.
Using 0.25 logFC  

In [ ]:
#---------- Seurat_Differential_Expression_p16
#----- Load data if needed
if (!exists("seurat_anno")) {
  seurat_anno <- readRDS(file = "seurat_results/seurat_anno.rds")
}

#----- RNA Assay stores the original and normalized counts
# Original counts : seurat_anno@assays[["RNA"]]@counts
# Normalized counts : seurat_anno@assays[["RNA"]]@data
DefaultAssay(seurat_anno) <- "RNA"
#----- Group Celltype ID
# make this the Idents()
metadata_p16 <- seurat_anno@meta.data %>%
  mutate(CDKN2A_Binary = case_when(CDKN2A_Count > 0 ~ "CDKN2A_Pos",
                                   CDKN2A_Count == 0 ~ "CDKN2A_Neg"))
seurat_anno@meta.data <- metadata_p16
rm(metadata_p16)
seurat_anno$group_cluster <- paste(seurat_anno$CDKN2A_Binary, 
                                   seurat_anno$cluster_annotation, 
                                   sep = "_")
Idents(seurat_anno) <- seurat_anno$group_cluster

#---------- Differential Expressed Genes
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Cell_Type Differential Expressed Genes
get_deg <- function(seurat_obj, cell_type, ident1, ident2){
  FindMarkers(seurat_obj, 
              ident.1 = paste(ident1, cell_type, sep = "_"), 
              ident.2 = paste(ident2, cell_type, sep = "_"), 
              test.use = "MAST", 
              latent.vars = "AGE", 
              logfc.threshold = 0, 
              min.pct = 0,
              min.diff.pct = -Inf, 
              min.cells.feature = 10, 
              min.cells.group = 10, 
              verbose = FALSE) %>% 
    rownames_to_column(var = "Gene_Symbol") %>% 
    left_join(Gene_Emsembl_Symbol, by="Gene_Symbol") %>% 
    mutate(Comparison = paste(ident1, "vs", ident2, cell_type, sep = "_"))
}
Differential_Expression <- lapply(cell_types, function(cell_type) {
  tryCatch(get_deg(seurat_obj = seurat_anno,
                   cell_type = cell_type,
                   ident1 = "CDKN2A_Pos",
                   ident2 = "CDKN2A_Neg"),
           error = function(e) 
             # empty dataframe with same column headers & types...
             data.frame(Gene_Symbol = character(),
                        p_val = double(),
                        avg_log2FC = double(),
                        pct.1 = double(),
                        pct.2 = double(),
                        p_val_adj = double(),
                        Gene_Emsembl = character(),
                        Comparison = character(),
                        stringsAsFactors = FALSE))
}) %>% bind_rows()
#----- save the results
write_delim(Differential_Expression, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN2A_Pos_vs_CDKN2A_Neg_AGE_MAST.txt"), 
  delim = "\t")
#----- Number of DEGs
Differential_Expression_count <- Differential_Expression %>% 
  mutate(Significant = case_when(
           p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
           p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
           TRUE ~ 'Not_Significant')) %>% 
  dplyr::count(Comparison, Significant) %>% 
  dplyr::rename(Count = n) %>% 
  tidyr::spread(Significant, Count)
#----- save the results
write_delim(Differential_Expression_count, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN2A_Pos_vs_CDKN2A_Neg_AGE_MAST_count.txt"), 
  delim = "\t")


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
Differential_Expression_GOBP <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  return(genes_go@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_GOBP, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN2A_Pos_vs_CDKN2A_Neg_AGE_MAST_Pathways_GSEA_GOBP.txt"), 
  delim = "\t")
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
Differential_Expression_REACTOME <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  return(genes_gsea@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_REACTOME, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN2A_Pos_vs_CDKN2A_Neg_AGE_MAST_Pathways_GSEA_REACTOME.txt"), 
  delim = "\t")

In [ ]:
#---------- Seurat_Differential_Expression_Figures_p16
Differential_Expression <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN2A_Pos_vs_CDKN2A_Neg_AGE_MAST.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)

#----- Differential_Expression_count
Differential_Expression_count <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN2A_Pos_vs_CDKN2A_Neg_AGE_MAST_count.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
Differential_Expression_count %>% 
  kable(caption="Number of Differentially Expressed Genes", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))


#---------- Differential Expressed Genes Volcano
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Differential Expressed Genes Volcano
for (comparison in comparisons){
  plot <- Differential_Expression %>% 
            dplyr::filter(Comparison == comparison) %>%
            dplyr::select(Comparison, 
                          Gene_Symbol, 
                          avg_log2FC, 
                          p_val,
                          p_val_adj) %>% 
            mutate(Significant = case_when(
                     p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
                     p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
                     TRUE ~ 'Not_Significant')) %>% 
            mutate(Label = case_when(
                     Significant == "Upregulated" ~ Gene_Symbol,
                     Significant == "Downregulated" ~ Gene_Symbol,
                     TRUE ~ NA_character_)) %>% 
            ggplot(mapping = aes(x=avg_log2FC, 
                   y = -log10(p_val), 
                   color = Significant,
                   label = Label)) + 
              geom_point(alpha = 0.5, stroke=0, size=2) + 
              geom_text_repel() +
              ggtitle(comparison) + 
              xlab("log2 Fold Change") +
              ylab("-log10(P Value)") + 
              theme_classic() + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold")) + 
              scale_colour_manual(values = c('Downregulated' = 'Blue',
                                             'Not_Significant' = 'Gray',
                                             'Upregulated' = 'Red')) + 
              geom_vline(xintercept = c(-0.25,0.25), 
                         linetype = "dotted")
  print(plot)
  # png(file = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
  #   width = 600*2, 
  #   height = 400*2,
  #   pointsize = 6)
  # print(plot)
  # dev.off()
  ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
         plot = plot,
         width = 600*2,
         height = 400*2,
         dpi = 150,
         units = "px")
}


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  if (nrow(genes_go@result) == 0) {
     warning(paste0("No significant GOBP genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
     plot <- enrichplot::dotplot(object = genes_go, 
                                  showCategory = 10, 
                                  split = ".sign") + 
                facet_grid(.~.sign) + 
                ggtitle(paste0(comparison, " - GO:Biological Process")) + 
                theme(plot.title = element_text(hjust = 0.5, 
                                                face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaGOBP.png", sep = "_"), 
          width=1000, 
          height=600)
      print(plot)
      dev.off()
   } 
  
  #----- Split GOBP Pathway Map by Activated / Suppressed
  # (otherwise the activated & suppressed are shown together -- misleading)
  genes_go_emapplot <- genes_go
  genes_go_emapplot@result <- genes_go_emapplot@result %>%
    mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                         NES < 0 ~ "suppressed"))
  genes_go_emapplot_activated <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "activated")
  genes_go_emapplot_suppressed <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "suppressed")
  
  # plot activated
  if (nrow(genes_go_emapplot_activated@result) == 0) {
     warning(paste0("No significant activated GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_activated <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_activated <- pairwise_termsim(x = genes_go_emapplot_activated)
    plot_activated <- genes_go_emapplot_activated %>% 
              enrichplot::emapplot(showCategory=40) +
              theme(plot.title = element_text(hjust = 0.5,
                                              face = "bold"))
    print(plot_activated)
  }
  
  # plot suppressed
  if (nrow(genes_go_emapplot_suppressed@result) == 0) {
     warning(paste0("No significant suppressed GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_suppressed <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_suppressed <- pairwise_termsim(x = genes_go_emapplot_suppressed)
    plot_suppressed <- genes_go_emapplot_suppressed %>% 
              enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
    print(plot_suppressed)
  } 
  
  # combine & save plot
  sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ncol = 2)
  
  
  plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                     " - GO:Biological Process"),
                                                     size = 24,
                                                     hjust = 0.5,
                                                     fontface = "bold"),
                             sub_plot_labels,
                             plot_grid(plot_activated, plot_suppressed),
                             ncol = 1, rel_heights = c(0.1, .05, 1))

  plot
  
  png(file=
    paste("seurat_results/figures/Seurat_Differential_Expression", 
           comparison, "AGE_MAST_Pathways_gseaGOBPenrich.png", sep = "_"), 
      width=1400, 
      height=800)
  print(plot)
  dev.off()
  #----- pathExplore: High-Level GSEA Clusters (GO:BP)
  tryCatch({
    # fix here: https://github.com/tidyverse/ggplot2/issues/2799
    cf <- coord_fixed()
    cf$default <- TRUE
    plot <- enrichmentNetwork(genes_go@result, drawEllipses = TRUE, fontSize = 2.5) +
      ggtitle(paste0(comparison, " - GO:Biological Process")) +
      theme(plot.title = element_text(hjust = 0.5, face = "bold"),
            plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
      cf +
      coord_fixed(clip = "off")
    print(plot)
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                            comparison, "AGE_MAST_Pathways_gseaGOBPenrich_pathExplore.png", 
                            sep = "_"),
           plot = plot,
           width = 600*2,
           height = 400*2,
           dpi = 150,
           units = "px")
  },
    error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
}
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  if (nrow(genes_gsea@result) == 0) {
    warning(paste0("No significant REACTOME genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
    plot <- enrichplot::dotplot(object = genes_gsea,
                                showCategory = 10, 
                                split = ".sign") + 
              facet_grid(.~.sign) + 
              ggtitle(paste0(comparison, " - GSEA Reactome")) + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold"))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME.png", sep = "_"), 
        width=1000, 
        height=600)
    print(plot)
    dev.off()
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME_quality.png", sep = "_"), 
       plot = plot,
       width = 1000*2,
       height = 600*2,
       dpi = 200,
       units = "px")
    
    #----- Split REACTOME Pathway Map by Activated / Suppressed
    # (otherwise the activated & suppressed are shown together -- misleading)
    genes_gsea_emapplot <- genes_gsea
    genes_gsea_emapplot@result <- genes_gsea_emapplot@result %>%
      mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                           NES < 0 ~ "suppressed"))
    genes_gsea_emapplot_activated <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "activated")
    genes_gsea_emapplot_suppressed <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "suppressed")
    
    # plot activated
    if (nrow(genes_gsea_emapplot_activated@result) == 0) {
       warning(paste0("No significant activated REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_activated <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_activated <- pairwise_termsim(x = genes_gsea_emapplot_activated)
      plot_activated <- genes_gsea_emapplot_activated %>% 
                enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
      print(plot_activated)
    }
    
    # plot suppressed
    if (nrow(genes_gsea_emapplot_suppressed@result) == 0) {
       warning(paste0("No significant suppressed REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_suppressed <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_suppressed <- pairwise_termsim(x = genes_gsea_emapplot_suppressed)
      plot_suppressed <- genes_gsea_emapplot_suppressed %>% 
                enrichplot::emapplot(showCategory=40) +
                  theme(plot.title = element_text(hjust = 0.5,
                                                  face = "bold"))
      print(plot_suppressed)
    } 
    
    # combine & save plot
    sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ncol = 2)
    
    
    plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                       " - GSEA Reactome"),
                                                       size = 24,
                                                       hjust = 0.5,
                                                       fontface = "bold"),
                               sub_plot_labels,
                               plot_grid(plot_activated, plot_suppressed),
                               ncol = 1, rel_heights = c(0.1, .05, 1))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich.png", sep = "_"), 
        width=1400, 
        height=800)
    print(plot)
    dev.off()
    #----- pathExplore: High-Level GSEA Clusters (GSEA Reactome)
    tryCatch({
      # fix here: https://github.com/tidyverse/ggplot2/issues/2799
      cf <- coord_fixed()
      cf$default <- TRUE
      plot <- pathExplore::enrichmentNetwork(genes_gsea@result, drawEllipses = TRUE, fontSize = 2.5) +
        ggtitle(paste0(comparison, " - GSEA Reactome")) +
        theme(plot.title = element_text(hjust = 0.5, face = "bold"),
              plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
        cf +
        coord_fixed(clip = "off")
      print(plot)
      ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                              comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich_pathExplore.png",
                              sep = "_"),
             plot = plot,
             width = 600*2,
             height = 400*2,
             dpi = 150,
             units = "px")
    },
      error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
   } 
}


## Seurat Differential Expression: p21+ / p21-

Find DEG between p21+ and p21- per cell_type
Using 0.25 logFC  

In [ ]:
#---------- Seurat_Differential_Expression_p21
#----- Load data if needed
if (!exists("seurat_anno")) {
  seurat_anno <- readRDS(file = "seurat_results/seurat_anno.rds")
}

#----- RNA Assay stores the original and normalized counts
# Original counts : seurat_anno@assays[["RNA"]]@counts
# Normalized counts : seurat_anno@assays[["RNA"]]@data
DefaultAssay(seurat_anno) <- "RNA"
#----- Group Celltype ID
# make this the Idents()
metadata_p21 <- seurat_anno@meta.data %>%
  mutate(CDKN1A_Binary = case_when(CDKN1A_Count > 0 ~ "CDKN1A_Pos",
                                   CDKN1A_Count == 0 ~ "CDKN1A_Neg"))
seurat_anno@meta.data <- metadata_p21
rm(metadata_p21)
seurat_anno$group_cluster <- paste(seurat_anno$CDKN1A_Binary, 
                                   seurat_anno$cluster_annotation, 
                                   sep = "_")
Idents(seurat_anno) <- seurat_anno$group_cluster

#---------- Differential Expressed Genes
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Cell_Type Differential Expressed Genes
get_deg <- function(seurat_obj, cell_type, ident1, ident2){
  FindMarkers(seurat_obj, 
              ident.1 = paste(ident1, cell_type, sep = "_"), 
              ident.2 = paste(ident2, cell_type, sep = "_"), 
              test.use = "MAST", 
              latent.vars = "AGE", 
              logfc.threshold = 0, 
              min.pct = 0,
              min.diff.pct = -Inf, 
              min.cells.feature = 10, 
              min.cells.group = 10, 
              verbose = FALSE) %>% 
    rownames_to_column(var = "Gene_Symbol") %>% 
    left_join(Gene_Emsembl_Symbol, by="Gene_Symbol") %>% 
    mutate(Comparison = paste(ident1, "vs", ident2, cell_type, sep = "_"))
}
Differential_Expression <- lapply(cell_types, function(cell_type) {
  tryCatch(get_deg(seurat_obj = seurat_anno,
                   cell_type = cell_type,
                   ident1 = "CDKN1A_Pos",
                   ident2 = "CDKN1A_Neg"),
           error = function(e) 
             # empty dataframe with same column headers & types...
             data.frame(Gene_Symbol = character(),
                        p_val = double(),
                        avg_log2FC = double(),
                        pct.1 = double(),
                        pct.2 = double(),
                        p_val_adj = double(),
                        Gene_Emsembl = character(),
                        Comparison = character(),
                        stringsAsFactors = FALSE))
}) %>% bind_rows()
#----- save the results
write_delim(Differential_Expression, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_Pos_vs_CDKN1A_Neg_AGE_MAST.txt"), 
  delim = "\t")
#----- Number of DEGs
Differential_Expression_count <- Differential_Expression %>% 
  mutate(Significant = case_when(
           p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
           p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
           TRUE ~ 'Not_Significant')) %>% 
  dplyr::count(Comparison, Significant) %>% 
  dplyr::rename(Count = n) %>% 
  tidyr::spread(Significant, Count)
#----- save the results
write_delim(Differential_Expression_count, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_Pos_vs_CDKN1A_Neg_AGE_MAST_count.txt"), 
  delim = "\t")


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
Differential_Expression_GOBP <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  return(genes_go@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_GOBP, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_Pos_vs_CDKN1A_Neg_AGE_MAST_Pathways_GSEA_GOBP.txt"), 
  delim = "\t")
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
Differential_Expression_REACTOME <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  return(genes_gsea@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_REACTOME, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_Pos_vs_CDKN1A_Neg_AGE_MAST_Pathways_GSEA_REACTOME.txt"), 
  delim = "\t")

In [ ]:
#---------- Seurat_Differential_Expression_Figures_p21
Differential_Expression <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_Pos_vs_CDKN1A_Neg_AGE_MAST.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)

#----- Differential_Expression_count
Differential_Expression_count <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_Pos_vs_CDKN1A_Neg_AGE_MAST_count.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
Differential_Expression_count %>% 
  kable(caption="Number of Differentially Expressed Genes", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))


#---------- Differential Expressed Genes Volcano
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Differential Expressed Genes Volcano
for (comparison in comparisons){
  plot <- Differential_Expression %>% 
            dplyr::filter(Comparison == comparison) %>%
            dplyr::select(Comparison, 
                          Gene_Symbol, 
                          avg_log2FC, 
                          p_val,
                          p_val_adj) %>% 
            mutate(Significant = case_when(
                     p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
                     p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
                     TRUE ~ 'Not_Significant')) %>% 
            mutate(Label = case_when(
                     Significant == "Upregulated" ~ Gene_Symbol,
                     Significant == "Downregulated" ~ Gene_Symbol,
                     TRUE ~ NA_character_)) %>% 
            ggplot(mapping = aes(x=avg_log2FC, 
                   y = -log10(p_val), 
                   color = Significant,
                   label = Label)) + 
              geom_point(alpha = 0.5, stroke=0, size=2) + 
              geom_text_repel() +
              ggtitle(comparison) + 
              xlab("log2 Fold Change") +
              ylab("-log10(P Value)") + 
              theme_classic() + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold")) + 
              scale_colour_manual(values = c('Downregulated' = 'Blue',
                                             'Not_Significant' = 'Gray',
                                             'Upregulated' = 'Red')) + 
              geom_vline(xintercept = c(-0.25,0.25), 
                         linetype = "dotted")
  print(plot)
  # png(file = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
  #   width = 600*2, 
  #   height = 400*2,
  #   pointsize = 6)
  # print(plot)
  # dev.off()
  ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
         plot = plot,
         width = 600*2,
         height = 400*2,
         dpi = 150,
         units = "px")
}


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  if (nrow(genes_go@result) == 0) {
     warning(paste0("No significant GOBP genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
     plot <- enrichplot::dotplot(object = genes_go, 
                                  showCategory = 10, 
                                  split = ".sign") + 
                facet_grid(.~.sign) + 
                ggtitle(paste0(comparison, " - GO:Biological Process")) + 
                theme(plot.title = element_text(hjust = 0.5, 
                                                face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaGOBP.png", sep = "_"), 
          width=1000, 
          height=600)
      print(plot)
      dev.off()
   } 
  
  #----- Split GOBP Pathway Map by Activated / Suppressed
  # (otherwise the activated & suppressed are shown together -- misleading)
  genes_go_emapplot <- genes_go
  genes_go_emapplot@result <- genes_go_emapplot@result %>%
    mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                         NES < 0 ~ "suppressed"))
  genes_go_emapplot_activated <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "activated")
  genes_go_emapplot_suppressed <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "suppressed")
  
  # plot activated
  if (nrow(genes_go_emapplot_activated@result) == 0) {
     warning(paste0("No significant activated GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_activated <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_activated <- pairwise_termsim(x = genes_go_emapplot_activated)
    plot_activated <- genes_go_emapplot_activated %>% 
              enrichplot::emapplot(showCategory=40) +
              theme(plot.title = element_text(hjust = 0.5,
                                              face = "bold"))
    print(plot_activated)
  }
  
  # plot suppressed
  if (nrow(genes_go_emapplot_suppressed@result) == 0) {
     warning(paste0("No significant suppressed GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_suppressed <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_suppressed <- pairwise_termsim(x = genes_go_emapplot_suppressed)
    plot_suppressed <- genes_go_emapplot_suppressed %>% 
              enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
    print(plot_suppressed)
  } 
  
  # combine & save plot
  sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ncol = 2)
  
  
  plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                     " - GO:Biological Process"),
                                                     size = 24,
                                                     hjust = 0.5,
                                                     fontface = "bold"),
                             sub_plot_labels,
                             plot_grid(plot_activated, plot_suppressed),
                             ncol = 1, rel_heights = c(0.1, .05, 1))

  plot
  
  png(file=
    paste("seurat_results/figures/Seurat_Differential_Expression", 
           comparison, "AGE_MAST_Pathways_gseaGOBPenrich.png", sep = "_"), 
      width=1400, 
      height=800)
  print(plot)
  dev.off()
  #----- pathExplore: High-Level GSEA Clusters (GO:BP)
  tryCatch({
    # fix here: https://github.com/tidyverse/ggplot2/issues/2799
    cf <- coord_fixed()
    cf$default <- TRUE
    plot <- enrichmentNetwork(genes_go@result, drawEllipses = TRUE, fontSize = 2.5) +
      ggtitle(paste0(comparison, " - GO:Biological Process")) +
      theme(plot.title = element_text(hjust = 0.5, face = "bold"),
            plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
      cf +
      coord_fixed(clip = "off")
    print(plot)
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                            comparison, "AGE_MAST_Pathways_gseaGOBPenrich_pathExplore.png", 
                            sep = "_"),
           plot = plot,
           width = 600*2,
           height = 400*2,
           dpi = 150,
           units = "px")
  },
    error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
}
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  if (nrow(genes_gsea@result) == 0) {
    warning(paste0("No significant REACTOME genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
    plot <- enrichplot::dotplot(object = genes_gsea,
                                showCategory = 10, 
                                split = ".sign") + 
              facet_grid(.~.sign) + 
              ggtitle(paste0(comparison, " - GSEA Reactome")) + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold"))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME.png", sep = "_"), 
        width=1000, 
        height=600)
    print(plot)
    dev.off()
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME_quality.png", sep = "_"), 
       plot = plot,
       width = 1000*2,
       height = 600*2,
       dpi = 200,
       units = "px")
    
    #----- Split REACTOME Pathway Map by Activated / Suppressed
    # (otherwise the activated & suppressed are shown together -- misleading)
    genes_gsea_emapplot <- genes_gsea
    genes_gsea_emapplot@result <- genes_gsea_emapplot@result %>%
      mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                           NES < 0 ~ "suppressed"))
    genes_gsea_emapplot_activated <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "activated")
    genes_gsea_emapplot_suppressed <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "suppressed")
    
    # plot activated
    if (nrow(genes_gsea_emapplot_activated@result) == 0) {
       warning(paste0("No significant activated REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_activated <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_activated <- pairwise_termsim(x = genes_gsea_emapplot_activated)
      plot_activated <- genes_gsea_emapplot_activated %>% 
                enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
      print(plot_activated)
    }
    
    # plot suppressed
    if (nrow(genes_gsea_emapplot_suppressed@result) == 0) {
       warning(paste0("No significant suppressed REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_suppressed <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_suppressed <- pairwise_termsim(x = genes_gsea_emapplot_suppressed)
      plot_suppressed <- genes_gsea_emapplot_suppressed %>% 
                enrichplot::emapplot(showCategory=40) +
                  theme(plot.title = element_text(hjust = 0.5,
                                                  face = "bold"))
      print(plot_suppressed)
    } 
    
    # combine & save plot
    sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ncol = 2)
    
    
    plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                       " - GSEA Reactome"),
                                                       size = 24,
                                                       hjust = 0.5,
                                                       fontface = "bold"),
                               sub_plot_labels,
                               plot_grid(plot_activated, plot_suppressed),
                               ncol = 1, rel_heights = c(0.1, .05, 1))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich.png", sep = "_"), 
        width=1400, 
        height=800)
    print(plot)
    dev.off()
    #----- pathExplore: High-Level GSEA Clusters (GSEA Reactome)
    tryCatch({
      # fix here: https://github.com/tidyverse/ggplot2/issues/2799
      cf <- coord_fixed()
      cf$default <- TRUE
      plot <- pathExplore::enrichmentNetwork(genes_gsea@result, drawEllipses = TRUE, fontSize = 2.5) +
        ggtitle(paste0(comparison, " - GSEA Reactome")) +
        theme(plot.title = element_text(hjust = 0.5, face = "bold"),
              plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
        cf +
        coord_fixed(clip = "off")
      print(plot)
      ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                              comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich_pathExplore.png",
                              sep = "_"),
             plot = plot,
             width = 600*2,
             height = 400*2,
             dpi = 150,
             units = "px")
    },
      error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
   } 
}


## Seurat Differential Expression: p16+ & p21+ Co-Expression

Find DEG between (p16+ & p21+) and (p16- or p21-) per cell_type
Using 0.25 logFC  

In [ ]:
#---------- Seurat_Differential_Expression_p16_p21
#----- Load data if needed
if (!exists("seurat_anno")) {
  seurat_anno <- readRDS(file = "seurat_results/seurat_anno.rds")
}

#----- RNA Assay stores the original and normalized counts
# Original counts : seurat_anno@assays[["RNA"]]@counts
# Normalized counts : seurat_anno@assays[["RNA"]]@data
DefaultAssay(seurat_anno) <- "RNA"
#----- Group Celltype ID
# make this the Idents()
metadata_p16_p21 <- seurat_anno@meta.data %>%
  mutate(CDKN1A_CDKN2A_Binary = case_when((CDKN1A_Count > 0) & (CDKN2A_Count > 0) ~ "CDKN1A_CDKN2A_Pos",
                                          (CDKN1A_Count == 0) | (CDKN2A_Count == 0) ~ "CDKN1A_or_CDKN2A_Neg"))
seurat_anno@meta.data <- metadata_p16_p21
rm(metadata_p16_p21)
seurat_anno$group_cluster <- paste(seurat_anno$CDKN1A_CDKN2A_Binary, 
                                   seurat_anno$cluster_annotation, 
                                   sep = "_")
Idents(seurat_anno) <- seurat_anno$group_cluster

#---------- Differential Expressed Genes
#----- Cell Types
cell_types <- seurat_anno@meta.data %>% 
  dplyr::arrange(cluster_annotation) %>% 
  dplyr::select(cluster_annotation) %>% 
  unique() %>% 
  pull(cluster_annotation)
#----- Cell_Type Differential Expressed Genes
get_deg <- function(seurat_obj, cell_type, ident1, ident2){
  FindMarkers(seurat_obj, 
              ident.1 = paste(ident1, cell_type, sep = "_"), 
              ident.2 = paste(ident2, cell_type, sep = "_"), 
              test.use = "MAST", 
              latent.vars = "AGE", 
              logfc.threshold = 0, 
              min.pct = 0,
              min.diff.pct = -Inf, 
              min.cells.feature = 10, 
              min.cells.group = 10, 
              verbose = FALSE) %>% 
    rownames_to_column(var = "Gene_Symbol") %>% 
    left_join(Gene_Emsembl_Symbol, by="Gene_Symbol") %>% 
    mutate(Comparison = paste(ident1, "vs", ident2, cell_type, sep = "_"))
}
Differential_Expression <- lapply(cell_types, function(cell_type) {
  tryCatch(get_deg(seurat_obj = seurat_anno,
                   cell_type = cell_type,
                   ident1 = "CDKN1A_CDKN2A_Pos",
                   ident2 = "CDKN1A_or_CDKN2A_Neg"),
           error = function(e) 
             # empty dataframe with same column headers & types...
             data.frame(Gene_Symbol = character(),
                        p_val = double(),
                        avg_log2FC = double(),
                        pct.1 = double(),
                        pct.2 = double(),
                        p_val_adj = double(),
                        Gene_Emsembl = character(),
                        Comparison = character(),
                        stringsAsFactors = FALSE))
}) %>% bind_rows()
#----- save the results
write_delim(Differential_Expression, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_CDKN2A_Pos_vs_CDKN1A_or_CDKN2A_Neg_AGE_MAST.txt"), 
  delim = "\t")
#----- Number of DEGs
Differential_Expression_count <- Differential_Expression %>% 
  mutate(Significant = case_when(
           p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
           p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
           TRUE ~ 'Not_Significant')) %>% 
  dplyr::count(Comparison, Significant) %>% 
  dplyr::rename(Count = n) %>% 
  tidyr::spread(Significant, Count)
#----- save the results
write_delim(Differential_Expression_count, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_CDKN2A_Pos_vs_CDKN1A_or_CDKN2A_Neg_AGE_MAST_count.txt"), 
  delim = "\t")


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
Differential_Expression_GOBP <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  return(genes_go@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_GOBP, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_CDKN2A_Pos_vs_CDKN1A_or_CDKN2A_Neg_AGE_MAST_Pathways_GSEA_GOBP.txt"), 
  delim = "\t")
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
Differential_Expression_REACTOME <- lapply(comparisons, function(comparison) {
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  return(genes_gsea@result %>% mutate(Comparison = comparison))
}) %>% bind_rows()
#----- Save Results
write_delim(Differential_Expression_REACTOME, 
  paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_CDKN2A_Pos_vs_CDKN1A_or_CDKN2A_Neg_AGE_MAST_Pathways_GSEA_REACTOME.txt"), 
  delim = "\t")

In [ ]:
#---------- Seurat_Differential_Expression_Figures_p16_p21
Differential_Expression <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_CDKN2A_Pos_vs_CDKN1A_or_CDKN2A_Neg_AGE_MAST.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)

#----- Differential_Expression_count
Differential_Expression_count <- 
  read_delim(paste0("seurat_results/", NameProject, "_Differential_Expression_CDKN1A_CDKN2A_Pos_vs_CDKN1A_or_CDKN2A_Neg_AGE_MAST_count.txt"), 
  delim = "\t", escape_double = FALSE, 
  trim_ws = TRUE)
Differential_Expression_count %>% 
  kable(caption="Number of Differentially Expressed Genes", 
        format.args = list(big.mark = ",")) %>% 
  kable_styling(c("striped", "bordered"))


#---------- Differential Expressed Genes Volcano
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Differential Expressed Genes Volcano
for (comparison in comparisons){
  plot <- Differential_Expression %>% 
            dplyr::filter(Comparison == comparison) %>%
            dplyr::select(Comparison, 
                          Gene_Symbol, 
                          avg_log2FC, 
                          p_val,
                          p_val_adj) %>% 
            mutate(Significant = case_when(
                     p_val_adj <= 0.05 & avg_log2FC >= 0.25 ~ 'Upregulated',
                     p_val_adj <= 0.05 & avg_log2FC <= -0.25 ~ 'Downregulated',
                     TRUE ~ 'Not_Significant')) %>% 
            mutate(Label = case_when(
                     Significant == "Upregulated" ~ Gene_Symbol,
                     Significant == "Downregulated" ~ Gene_Symbol,
                     TRUE ~ NA_character_)) %>% 
            ggplot(mapping = aes(x=avg_log2FC, 
                   y = -log10(p_val), 
                   color = Significant,
                   label = Label)) + 
              geom_point(alpha = 0.5, stroke=0, size=2) + 
              geom_text_repel() +
              ggtitle(comparison) + 
              xlab("log2 Fold Change") +
              ylab("-log10(P Value)") + 
              theme_classic() + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold")) + 
              scale_colour_manual(values = c('Downregulated' = 'Blue',
                                             'Not_Significant' = 'Gray',
                                             'Upregulated' = 'Red')) + 
              geom_vline(xintercept = c(-0.25,0.25), 
                         linetype = "dotted")
  print(plot)
  # png(file = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
  #   width = 600*2, 
  #   height = 400*2,
  #   pointsize = 6)
  # print(plot)
  # dev.off()
  ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", comparison, "AGE_MAST_volcano.png", sep = "_"),
         plot = plot,
         width = 600*2,
         height = 400*2,
         dpi = 150,
         units = "px")
}


#---------- Differential Expressed Genes Pathways
#----- Comparisons
comparisons <- Differential_Expression$Comparison %>% 
                 unique()
#----- Pathways GSEA GOBP
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_go <- gseGO(geneList = genelist, 
                    ont ="BP", 
                    OrgDb = get("org.Hs.eg.db"),
                    keyType = "SYMBOL", 
                    minGSSize = 10, 
                    maxGSSize = 500, 
                    pvalueCutoff = 0.05, 
                    pAdjustMethod = "BH",
                    verbose = TRUE)
  if (nrow(genes_go@result) == 0) {
     warning(paste0("No significant GOBP genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
     plot <- enrichplot::dotplot(object = genes_go, 
                                  showCategory = 10, 
                                  split = ".sign") + 
                facet_grid(.~.sign) + 
                ggtitle(paste0(comparison, " - GO:Biological Process")) + 
                theme(plot.title = element_text(hjust = 0.5, 
                                                face = "bold"))
      print(plot)
      png(file=
        paste("seurat_results/figures/Seurat_Differential_Expression", 
               comparison, "AGE_MAST_Pathways_gseaGOBP.png", sep = "_"), 
          width=1000, 
          height=600)
      print(plot)
      dev.off()
   } 
  
  #----- Split GOBP Pathway Map by Activated / Suppressed
  # (otherwise the activated & suppressed are shown together -- misleading)
  genes_go_emapplot <- genes_go
  genes_go_emapplot@result <- genes_go_emapplot@result %>%
    mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                         NES < 0 ~ "suppressed"))
  genes_go_emapplot_activated <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "activated")
  genes_go_emapplot_suppressed <- dplyr::filter(genes_go_emapplot, 
                                               pathway_direction == "suppressed")
  
  # plot activated
  if (nrow(genes_go_emapplot_activated@result) == 0) {
     warning(paste0("No significant activated GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_activated <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_activated <- pairwise_termsim(x = genes_go_emapplot_activated)
    plot_activated <- genes_go_emapplot_activated %>% 
              enrichplot::emapplot(showCategory=40) +
              theme(plot.title = element_text(hjust = 0.5,
                                              face = "bold"))
    print(plot_activated)
  }
  
  # plot suppressed
  if (nrow(genes_go_emapplot_suppressed@result) == 0) {
     warning(paste0("No significant suppressed GOBP genes for: ", 
                    comparison, "\nSkipping to next comparison..."))
    plot_suppressed <- ggplot() + theme_void()
  } else {
    genes_go_emapplot_suppressed <- pairwise_termsim(x = genes_go_emapplot_suppressed)
    plot_suppressed <- genes_go_emapplot_suppressed %>% 
              enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
    print(plot_suppressed)
  } 
  
  # combine & save plot
  sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                       size = 18,
                                                                       hjust = 0.5,
                                                                       fontface = "bold"),
                                        ncol = 2)
  
  
  plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                     " - GO:Biological Process"),
                                                     size = 24,
                                                     hjust = 0.5,
                                                     fontface = "bold"),
                             sub_plot_labels,
                             plot_grid(plot_activated, plot_suppressed),
                             ncol = 1, rel_heights = c(0.1, .05, 1))

  plot
  
  png(file=
    paste("seurat_results/figures/Seurat_Differential_Expression", 
           comparison, "AGE_MAST_Pathways_gseaGOBPenrich.png", sep = "_"), 
      width=1400, 
      height=800)
  print(plot)
  dev.off()
  #----- pathExplore: High-Level GSEA Clusters (GO:BP)
  tryCatch({
    # fix here: https://github.com/tidyverse/ggplot2/issues/2799
    cf <- coord_fixed()
    cf$default <- TRUE
    plot <- enrichmentNetwork(genes_go@result, drawEllipses = TRUE, fontSize = 2.5) +
      ggtitle(paste0(comparison, " - GO:Biological Process")) +
      theme(plot.title = element_text(hjust = 0.5, face = "bold"),
            plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
      cf +
      coord_fixed(clip = "off")
    print(plot)
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                            comparison, "AGE_MAST_Pathways_gseaGOBPenrich_pathExplore.png", 
                            sep = "_"),
           plot = plot,
           width = 600*2,
           height = 400*2,
           dpi = 150,
           units = "px")
  },
    error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
}
#----- Reactome
reactome <- msigdbr(species = "Homo sapiens", 
                    category = "C2", 
                    subcategory = "CP:REACTOME") %>% 
              dplyr::select(gs_name, gene_symbol)
#----- Pathways GSEA REACTOME
for (comparison in comparisons){
  genes <- Differential_Expression %>% 
    dplyr::filter(Comparison == comparison) %>%
    dplyr::select(Comparison, 
                  Gene_Symbol, 
                  avg_log2FC, 
                  p_val) %>%
    dplyr::mutate(p_val = case_when(p_val == 0 ~ 5e-324,
                                    TRUE ~ p_val)) %>%
    dplyr::mutate(sign_log10Pvalue = sign(avg_log2FC) * -log10(p_val)) %>% 
    dplyr::arrange(sign_log10Pvalue)
  genelist <- setNames(genes$sign_log10Pvalue, 
                       genes$Gene_Symbol) %>% 
    sort(decreasing = TRUE)
  genes_gsea <- GSEA(geneList = genelist,
                     minGSSize = 10,
                     maxGSSize = 500,
                     pvalueCutoff = 0.05,
                     pAdjustMethod = "BH",
                     TERM2GENE = reactome, 
                     by = "fgsea",
                     verbose = TRUE)
  if (nrow(genes_gsea@result) == 0) {
    warning(paste0("No significant REACTOME genes for: ", comparison, "\nSkipping to next comparison..."))
   } else {
    plot <- enrichplot::dotplot(object = genes_gsea,
                                showCategory = 10, 
                                split = ".sign") + 
              facet_grid(.~.sign) + 
              ggtitle(paste0(comparison, " - GSEA Reactome")) + 
              theme(plot.title = element_text(hjust = 0.5, 
                                              face = "bold"))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME.png", sep = "_"), 
        width=1000, 
        height=600)
    print(plot)
    dev.off()
    ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOME_quality.png", sep = "_"), 
       plot = plot,
       width = 1000*2,
       height = 600*2,
       dpi = 200,
       units = "px")
    
    #----- Split REACTOME Pathway Map by Activated / Suppressed
    # (otherwise the activated & suppressed are shown together -- misleading)
    genes_gsea_emapplot <- genes_gsea
    genes_gsea_emapplot@result <- genes_gsea_emapplot@result %>%
      mutate(pathway_direction = case_when(NES > 0 ~ "activated",
                                           NES < 0 ~ "suppressed"))
    genes_gsea_emapplot_activated <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "activated")
    genes_gsea_emapplot_suppressed <- dplyr::filter(genes_gsea_emapplot, 
                                                 pathway_direction == "suppressed")
    
    # plot activated
    if (nrow(genes_gsea_emapplot_activated@result) == 0) {
       warning(paste0("No significant activated REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_activated <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_activated <- pairwise_termsim(x = genes_gsea_emapplot_activated)
      plot_activated <- genes_gsea_emapplot_activated %>% 
                enrichplot::emapplot(showCategory=40) +
                theme(plot.title = element_text(hjust = 0.5,
                                                face = "bold"))
      print(plot_activated)
    }
    
    # plot suppressed
    if (nrow(genes_gsea_emapplot_suppressed@result) == 0) {
       warning(paste0("No significant suppressed REACTOME genes for: ", 
                      comparison, "\nSkipping to next comparison..."))
      plot_suppressed <- ggplot() + theme_void()
    } else {
      genes_gsea_emapplot_suppressed <- pairwise_termsim(x = genes_gsea_emapplot_suppressed)
      plot_suppressed <- genes_gsea_emapplot_suppressed %>% 
                enrichplot::emapplot(showCategory=40) +
                  theme(plot.title = element_text(hjust = 0.5,
                                                  face = "bold"))
      print(plot_suppressed)
    } 
    
    # combine & save plot
    sub_plot_labels <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = "ACTIVATED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ggdraw() + cowplot::draw_label(label = "SUPPRESSED",
                                                                         size = 18,
                                                                         hjust = 0.5,
                                                                         fontface = "bold"),
                                          ncol = 2)
    
    
    plot <- cowplot::plot_grid(ggdraw() + cowplot::draw_label(label = paste0(comparison, 
                                                       " - GSEA Reactome"),
                                                       size = 24,
                                                       hjust = 0.5,
                                                       fontface = "bold"),
                               sub_plot_labels,
                               plot_grid(plot_activated, plot_suppressed),
                               ncol = 1, rel_heights = c(0.1, .05, 1))
    print(plot)
    png(file=
      paste("seurat_results/figures/Seurat_Differential_Expression", 
             comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich.png", sep = "_"), 
        width=1400, 
        height=800)
    print(plot)
    dev.off()
    #----- pathExplore: High-Level GSEA Clusters (GSEA Reactome)
    tryCatch({
      # fix here: https://github.com/tidyverse/ggplot2/issues/2799
      cf <- coord_fixed()
      cf$default <- TRUE
      plot <- pathExplore::enrichmentNetwork(genes_gsea@result, drawEllipses = TRUE, fontSize = 2.5) +
        ggtitle(paste0(comparison, " - GSEA Reactome")) +
        theme(plot.title = element_text(hjust = 0.5, face = "bold"),
              plot.margin = margin(l = 6, t = 12, r = 3, unit = "pt")) +
        cf +
        coord_fixed(clip = "off")
      print(plot)
      ggsave(filename = paste("seurat_results/figures/Seurat_Differential_Expression",
                              comparison, "AGE_MAST_Pathways_gseaREACTOMEenrich_pathExplore.png",
                              sep = "_"),
             plot = plot,
             width = 600*2,
             height = 400*2,
             dpi = 150,
             units = "px")
    },
      error=function(e){cat("ERROR :",conditionMessage(e), "\n")})
   } 
}
